readme before running the code:


1.   Make sure the colab use GPU
2.   To run the ablation study section, please run the previous sections till the end of Best model struture. Then please run the first cell of Ablation study to ensure the device, params and indicator setting.



In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1QLcGf_KvHW_03h9iKnGJGG2NYPwPTKEz'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')  

id = '1-pif5yJFkaA8bu7_EH9Nysvi7X17ydWV'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('val.csv')  

id = '1zikGjn6r2B1vQZNOsNL9fLg6Q8A9g0Yo'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_without_labels.csv')

id = '1b0isFShOUPoyIkKrvi86Iq7D3FTbDj_Y'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('chat.csv')  

id = '10MzlRT69-pfaGX4i695EjjpQxWkDn2JO'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('item_id.csv')

id = '182oB9yA9IMvr1X3S944ZW2bd2-pMzt72'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('heros.csv')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import pandas as pd
import numpy as np
import torch
from torch import nn
from matplotlib import pyplot as plt
from io import StringIO
import time
from tqdm import tqdm
import torch
from torchvision import datasets, transforms
import gc

# !pip install GPUtil
# from GPUtil import showUtilization as gpu_usage
from sklearn.metrics import classification_report, f1_score

# Load File

In [ ]:
FILENAME_TRAIN = "/content/train.csv"
FILENAME_VAL = "/content/val.csv"
FILENAME_TEST = "/content/test_without_labels.csv"

In [ ]:
train = pd.read_csv(FILENAME_TRAIN)
val = pd.read_csv(FILENAME_VAL)
test = pd.read_csv(FILENAME_TEST)

In [ ]:
train

,sents,labels
0,wow,O
1,WTF,T
2,wpe wpe,O O
3,hahaha,O
4,wtf,T
...,...,...
26073,i bet,P O
26074,cant believe it,O O P
26075,AH,O
26076,dayuuuuuum [SEPA] lool,O SEPA O


# Preprocess

In [ ]:
def preprocess(dataset, is_train = True):
    # Step 1: split sents and labels to tokens and corresponding tagging
    dataset["sents"] = dataset["sents"].apply(lambda x: x.lower())
    dataset["sents"] =  dataset["sents"].apply(lambda x: x.split())
    if is_train:
      dataset["labels"] =  dataset["labels"].apply(lambda x: x.split())
    # Step 2: assert each row in sents and labels token have the same length
      for i in range(dataset.shape[0]):
          assert(dataset["sents"] is not None)
          assert(len(dataset["sents"][i]) == len(dataset["labels"][i]))
      x = dataset["sents"]
      y = dataset["labels"]
      return x, y
    else:
      x = dataset["sents"]
    return x

x_train, y_train = preprocess(train, is_train = True)
x_val, y_val = preprocess(val, is_train = True)
x_test = preprocess(test, is_train = False)
print(x_test)

0                                               [fucker]
1                                                [hahha]
2                                                [ggggg]
3                                            [macropyre]
4                                                 [boom]
                             ...                        
495    [report, spectre, rage, quit, [sepa], and, him...
496                             [sf, feeder, auto, lose]
497                      [gg, [sepa], commend, supports]
498    [how, [sepa], wtf, [sepa], this, is, not, even...
499    [coming, edi, wait, awhile, ya, [sepa], fuck, me]
Name: sents, Length: 500, dtype: object


## WordList and TagList

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG: 0, STOP_TAG: 1, 'T': 2, 'C': 3, 'D': 4, 'S': 5, 'P': 6, 'O': 7, 'SEPA': 8}
ix_to_tag = {v: k for k, v in tag_to_ix.items()}

In [ ]:
# Construct word list, word index
def index(x):
    # x: all training sentences
    word_set = set() 
    for element in x:
        for word in element:
            word_set.add(word)

    word_list = list(word_set) 
    word_list.sort()
    
    word_index = {}
    ind = 0
    for word in word_list:
        word_index[word] = ind
        ind += 1
    inv_map = {v: k for k, v in word_index.items()}
    return word_list, word_index, inv_map
  
all_word = x_train.to_list() + x_val.to_list() + x_test.to_list()
word_list, word_to_ix, ix_to_word = index(all_word)
word_to_ix

{'#error!': 0,
 '.': 1,
 '0': 2,
 '00': 3,
 '000': 4,
 '000000': 5,
 '000000000000000': 6,
 '00100': 7,
 '0162282307': 8,
 '025': 9,
 '040': 10,
 '06': 11,
 '0:00': 12,
 '0coins': 13,
 '0o': 14,
 '0v5': 15,
 '0ward': 16,
 '1': 17,
 '1.': 18,
 '10': 19,
 '100': 20,
 '1000': 21,
 '10000': 22,
 '100000': 23,
 '100hp': 24,
 '101': 25,
 '10120': 26,
 '102': 27,
 '103': 28,
 '10k': 29,
 '10min': 30,
 '10mins': 31,
 '10pm': 32,
 '10x': 33,
 '11': 34,
 '110': 35,
 '1110': 36,
 '113': 37,
 '113924617': 38,
 '115': 39,
 '1150': 40,
 '118': 41,
 '1188': 42,
 '119': 43,
 '11mins': 44,
 '11pm': 45,
 '12': 46,
 '120s': 47,
 '12k': 48,
 '12mins': 49,
 '12y': 50,
 '13': 51,
 '130cannot': 52,
 '134': 53,
 '1389461397419': 54,
 '13:8': 55,
 '13mins': 56,
 '14': 57,
 '1400': 58,
 '143': 59,
 '1450': 60,
 '149': 61,
 '15': 62,
 '150': 63,
 '1500': 64,
 '15200': 65,
 '1584': 66,
 '15min': 67,
 '15y': 68,
 '16': 69,
 '1600': 70,
 '1651': 71,
 '17': 72,
 '175': 73,
 '179': 74,
 '18': 75,
 '180': 76,
 '185': 

## Encode Data and Lebel to Index

In [ ]:
def to_index(data, to_ix):
    """
    Transform x or y data to list of index based on word_to_index or tag to index
    """
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(x_train, word_to_ix)
train_output_index = to_index(y_train,tag_to_ix)
val_input_index = to_index(x_val, word_to_ix)
val_output_index = to_index(y_val,tag_to_ix)
test_input_index = to_index(x_test,word_to_ix)

In [ ]:
train_input_index[1:10]

[[10976],
 [10930, 10930],
 [4478],
 [10976],
 [4946, 1960, 542, 7574, 542, 10809, 269, 10018],
 [1483],
 [0],
 [4156, 542, 8211, 6663, 9617, 8039, 542, 7593],
 [3499, 6364]]

## External Dataset

### Chat

In [ ]:
FILENAME_CHAT = "/content/chat.csv"
# obtain the required chat data
dota_chat = pd.read_csv(FILENAME_CHAT).iloc[:,1:3]
dota_chat['key1'] = (dota_chat['slot'] != dota_chat['slot'].shift(1)).astype(int).cumsum()
dota_chat['key'] = dota_chat['key'].astype('str')
dota_chat.groupby(['key1', 'slot'])['key'].apply(' '.join)
chat_list = []
# cleaning for the chat data
for row in dota_chat.iloc[:,0]:
  row_clean = re.sub(r"[^a-z0-9]+", ' ', row)
  row_clean = re.sub(r'[0-9]+', ' ', row_clean)
  row_clean = row_clean.lower()
  row_split = row_clean.split(' ')
  chat_list.append(row_split)

In [ ]:
# remove empty element
for i in chat_list:
  while("" in i) :
    i.remove("")
chat_list_clean = [i for i in chat_list if i != []]
chat_list_clean

[['force', 'it'],
 ['space', 'created'],
 ['hah'],
 ['ez'],
 ['mvp', 'ulti'],
 ['bye'],
 ['hah'],
 ['fate'],
 ['is', 'cruel'],
 ['fuck', 'my', 'ass'],
 ['ka', 'bu', 'tooooooooooooo'],
 ['wtf'],
 ['u', 'srsly'],
 ['sad', 'spec'],
 ['noes', 'cape', 'for', 'him'],
 ['wat'],
 ['that', 'one', 'i', 'cant', 'even', 'run'],
 ['why', 'alyway', 'hit', 'me'],
 ['what', 'did', 'i', 'do'],
 ['everyone', 'kiting', 'me'],
 ['u', 'stand', 'in', 'front', 'of', 'me'],
 ['cause', 'u'],
 ['fuck', 'yeah'],
 ['l'],
 ['why', 'so', 'rude'],
 ['tot', 'this', 'casual', 'banter'],
 ['weahaha'],
 ['wahaha'],
 ['hah'],
 ['space', 'boys'],
 ['haha'],
 ['hah'],
 ['hah'],
 ['we', 'are', 'losing'],
 ['hahah'],
 ['alche'],
 ['hahahha'],
 ['but', 'we', 'dive', 'much', 'any', 'other'],
 ['u', 'guys', 'are', 'lucky'],
 ['u', 'dont', 'have', 'a', 'k', 'mmr', 'player', 'with', 'u'],
 ['lol'],
 ['wlwlwlwllww'],
 ['torso', 'highest', 'here'],
 ['really'],
 ['wahaha'],
 ['g'],
 ['k', 'slayer'],
 ['haha'],
 ['no', 'k'],
 ['no',

In [ ]:
# remove empty element and set length
MAX_LEN = 10
chat_list_clean_len = []
for chat in chat_list:
  clean_chat = []
  for word in chat:
      if len(word) <= MAX_LEN and len(word) > 0:
          clean_chat.append(word)
  if len(clean_chat) != 0:
    chat_list_clean_len.append(clean_chat)

In [ ]:
domain_list_chat, domain_to_ix_chat, ix_to_domain = index(chat_list_clean_len)
domain_to_ix_chat

{'a': 0,
 'aa': 1,
 'aaa': 2,
 'aaaa': 3,
 'aaaaa': 4,
 'aaaaaa': 5,
 'aaaaaaaa': 6,
 'aaaaaaaaa': 7,
 'aaaaaaaaaa': 8,
 'aaaaaaaah': 9,
 'aaaaaaaall': 10,
 'aaaaaaaand': 11,
 'aaaaaaaggg': 12,
 'aaaaaaah': 13,
 'aaaaaah': 14,
 'aaaaaanal': 15,
 'aaaaaand': 16,
 'aaaaack': 17,
 'aaaaad': 18,
 'aaaaanal': 19,
 'aaaaaye': 20,
 'aaaaayep': 21,
 'aaaaayyyy': 22,
 'aaaad': 23,
 'aaaah': 24,
 'aaaand': 25,
 'aaaasss': 26,
 'aaaawn': 27,
 'aaaay': 28,
 'aaaayyyy': 29,
 'aaaayyyyyy': 30,
 'aaaedoom': 31,
 'aaah': 32,
 'aaaha': 33,
 'aaahahahah': 34,
 'aaahha': 35,
 'aaahhh': 36,
 'aaahhhh': 37,
 'aaahhhhh': 38,
 'aaain': 39,
 'aaait': 40,
 'aaaj': 41,
 'aaajajaj': 42,
 'aaand': 43,
 'aaaq': 44,
 'aaaw': 45,
 'aaax': 46,
 'aaay': 47,
 'aaaye': 48,
 'aab': 49,
 'aaba': 50,
 'aabn': 51,
 'aabscum': 52,
 'aabut': 53,
 'aacd': 54,
 'aack': 55,
 'aacount': 56,
 'aade': 57,
 'aadu': 58,
 'aaegis': 59,
 'aaese': 60,
 'aafk': 61,
 'aafter': 62,
 'aag': 63,
 'aaga': 64,
 'aagaaha': 65,
 'aagaga': 66,
 '

### Dota Glossary

In [ ]:
 # crawl for external data glossary
import requests
import bs4
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = list(set(stopwords.words('english')))

url = "https://dota2.fandom.com/wiki/Glossary"
response = requests.get(url)
html = response.text
soup = bs4.BeautifulSoup(html, "html.parser")
element_lst = []
for element in soup.find_all("dt"):
     text = element.text.lower()
     word = text.replace(u'\xa0', '')
     if '/' in word:
         element_lst.extend(re.split(r'/', word))
     elif 'or' in word:
         word = word.replace('or', '')
         word = re.sub(r'[^\w\s]','', word)
         element_lst.extend(word.split())
     else:
       element_lst.append(word)

heros_file = pd.read_csv("/content/heros.csv")
items_file = pd.read_csv("/content/item_id.csv") 
all_heros = heros_file['localized_name'].apply(lambda x: x.lower())
heros = all_heros.to_list()
all_hero_words= []
for x in heros:
     for word in x.split():
         if word not in stopwords and word.isdigit() == False:
           all_hero_words.append(word)

items = items_file['item_name'].apply(lambda x: re.sub(r'_',' ', x))
items = items.to_list()
all_items_word = []
for x in items:
     for word in x.split():
         if word not in stopwords and word.isdigit() == False:
           all_items_word.append(word)
domain_word_glossary = list(set(element_lst+all_hero_words+all_items_word))
domain_word_glossary.sort()
ind = 0
domain_to_ix_glossary = {}
for word in domain_word_glossary:
     domain_to_ix_glossary[word] = ind
     ind += 1
domain_to_ix_glossary

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'aa': 0,
 'abaddon': 1,
 'ability': 2,
 'abyssal': 3,
 'ac': 4,
 'aegis': 5,
 'aether': 6,
 'aggro': 7,
 'alacrity': 8,
 'alchemist': 9,
 'am': 10,
 'amulet': 11,
 'ancient': 12,
 'anti-mage': 13,
 'aoe': 14,
 'apparition': 15,
 'aquila': 16,
 'arc': 17,
 'arcane': 18,
 'armlet': 19,
 'arrow': 20,
 'assassin': 21,
 'assault': 22,
 'atos': 23,
 'attack': 24,
 'aw': 25,
 'axe': 26,
 'b': 27,
 'backdo': 28,
 'ball': 29,
 'banana': 30,
 'band': 31,
 'bane': 32,
 'bar': 33,
 'bara, barathum': 34,
 'barracks': 35,
 'basher': 36,
 'basi': 37,
 'basilius': 38,
 'bat': 39,
 'batrider': 40,
 'bb': 41,
 'bd': 42,
 'beastmaster': 43,
 'belt': 44,
 'bf': 45,
 'bfury': 46,
 'bh': 47,
 'bird': 48,
 'bkb': 49,
 'black': 50,
 'blade': 51,
 'blades': 52,
 'blight': 53,
 'blink': 54,
 'blocking': 55,
 'bloodseeker': 56,
 'bloodstone': 57,
 'bloodthorn': 58,
 'bm': 59,
 'book': 60,
 'booster': 61,
 'boots': 62,
 'bot': 63,
 'bot, bots': 64,
 'bottle': 65,
 'bottle crow': 66,
 'bounty': 67,
 'bracer': 68,

# Embedding

## PoS tag

In [ ]:
import nltk
from nltk import pos_tag
# information for pos tag: https://cs.nyu.edu/~grishman/jet/guide/PennPOS.html
# https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
unique_tags = sorted(list(nltk.load('help/tagsets/upenn_tagset.pickle').keys()))
print(unique_tags)
tag_index = {}
ind = 0
for tag in list(unique_tags):
    tag_index[tag] = ind
    ind += 1

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
['$', "''", '(', ')', ',', '--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [ ]:
len(tag_index)
tag_index

{'$': 0,
 "''": 1,
 '(': 2,
 ')': 3,
 ',': 4,
 '--': 5,
 '.': 6,
 ':': 7,
 'CC': 8,
 'CD': 9,
 'DT': 10,
 'EX': 11,
 'FW': 12,
 'IN': 13,
 'JJ': 14,
 'JJR': 15,
 'JJS': 16,
 'LS': 17,
 'MD': 18,
 'NN': 19,
 'NNP': 20,
 'NNPS': 21,
 'NNS': 22,
 'PDT': 23,
 'POS': 24,
 'PRP': 25,
 'PRP$': 26,
 'RB': 27,
 'RBR': 28,
 'RBS': 29,
 'RP': 30,
 'SYM': 31,
 'TO': 32,
 'UH': 33,
 'VB': 34,
 'VBD': 35,
 'VBG': 36,
 'VBN': 37,
 'VBP': 38,
 'VBZ': 39,
 'WDT': 40,
 'WP': 41,
 'WP$': 42,
 'WRB': 43,
 '``': 44}

## Char-Level Embedding

In [ ]:
char_list =[]
char_to_idx = {}
for sentence in all_word:
    for word in sentence:
        for char in word:
          if char not in char_list:
              char_list.append(char)
char_list.sort()
char_list

char_to_idx = {}
ind = 0
for char in char_list:
    char_to_idx[char] = ind
    ind += 1
char_to_idx

{'!': 0,
 '#': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 '*': 5,
 '-': 6,
 '.': 7,
 '/': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '<': 21,
 '=': 22,
 '>': 23,
 '@': 24,
 '[': 25,
 '\\': 26,
 ']': 27,
 '^': 28,
 '_': 29,
 '`': 30,
 'a': 31,
 'b': 32,
 'c': 33,
 'd': 34,
 'e': 35,
 'f': 36,
 'g': 37,
 'h': 38,
 'i': 39,
 'j': 40,
 'k': 41,
 'l': 42,
 'm': 43,
 'n': 44,
 'o': 45,
 'p': 46,
 'q': 47,
 'r': 48,
 's': 49,
 't': 50,
 'u': 51,
 'v': 52,
 'w': 53,
 'x': 54,
 'y': 55,
 'z': 56,
 '{': 57,
 '|': 58,
 '\ue006': 59}

In [ ]:
# convert input to character-level index
train_input_char_index = [to_index(x, char_to_idx) for x in x_train]
val_input_char_index = [to_index(x, char_to_idx) for x in x_val]
test_input_char_index = [to_index(x, char_to_idx) for x in x_test]

## Embedding Table Generation

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
def pos_embedding_table(tag_index, word_list):
      """
      The function constructs one-hot pos-tag embeddings.
      """
      # the function is to construct pos tag embeddings
      embedding_matrix = []
      for word, tag in nltk.pos_tag(word_list):
          embeddings = [0] * len(tag_index)
          embeddings[tag_index[tag]] = 1
          embedding_matrix.append(embeddings)
      embedding_matrix = np.array(embedding_matrix)
      return embedding_matrix

def word_embedding_table(word_list, pretrained_50 = False, pretrained_100 = False, word2vec_cbow = False, word2vec_sg = False, fasttext_cbow = False, fasttext_sg = False, concate = False):
      if concate == True:
        # concate pretrained glove-twitter-100 embedding & fastText embedding
        word_emb_model_glove = api.load("glove-twitter-100") 
        word_emb_model_fast_sg = FastText(sentences=all_word, size=100, window=5, min_count=1, min_n = 2, workers=4, sg=1)
        EMBEDDING_DIM = 200
        embedding_matrix = []
        for word in word_list:
            try:
                embedding_matrix.append(np.concatenate((word_emb_model_fast_sg.wv[word], word_emb_model_glove[word]),0))
            except:
                embedding_matrix.append([0]*EMBEDDING_DIM)
        embedding_matrix = np.array(embedding_matrix)
        return embedding_matrix

      else:
        # embedding from single model
        EMBEDDING_DIM = 100
        if pretrained_50 == True:
          word_emb_model = api.load("glove-twitter-50")
          EMBEDDING_DIM = 50 
        if pretrained_100 == True:
          word_emb_model = api.load("glove-twitter-100") 
        if word2vec_cbow == True:
          word_emb_model = Word2Vec(sentences=all_word, size=100, window=5, min_count=1, workers=4, sg=0)
        if word2vec_sg == True:
          word_emb_model = Word2Vec(sentences=all_word, size=100, window=5, min_count=1, workers=4, sg=1)
        if fasttext_cbow == True:
          word_emb_model = FastText(sentences=all_word, size=100, window=5, min_count=1, min_n = 2, workers=4, sg=0)
        if fasttext_sg == True:
          word_emb_model = FastText(sentences=all_word, size=100, window=5, min_count=1, min_n = 2, workers=4, sg=1)

        embedding_matrix = []
        for word in word_list:
            try:
                if pretrained_50 == True or pretrained_100 == True:
                  embedding_matrix.append(word_emb_model[word])
                else:
                  embedding_matrix.append(word_emb_model.wv[word])
            except:
                embedding_matrix.append([0]*EMBEDDING_DIM)
        embedding_matrix = np.array(embedding_matrix)
        return embedding_matrix

def domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = True, concate = False):
  if external == True:
    # train with only domain data (external chat)
    word_emb_model = FastText(sentences=domain_data_chat, size=100, window=5, min_count=1, min_n = 2, workers=4, sg=1)
    EMBEDDING_DIM = 100
    embedding_matrix = []
    for word in word_list:
      try:
        embedding_matrix.append(word_emb_model.wv[word])
      except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
    embedding_matrix = np.array(embedding_matrix)

  elif concate == True:
    # train with external chat + dota domain glossary data
    word_emb_model = FastText(sentences=domain_data_chat, size=100, window=5, min_count=1, min_n = 2, workers=4, sg=1)
    EMBEDDING_DIM = 100+max(domain_to_ix_glossary.values())
    embedding_matrix = []
    for word in word_list:
      if word in word_emb_model.wv and word in list(domain_to_ix_glossary.keys()):
        embeddings = [0] * max(domain_to_ix_glossary.values())
        embeddings[domain_to_ix_glossary[word]] = 1
        embedding_matrix.append(np.concatenate((word_emb_model.wv[word], np.array(embeddings)),0))
      else:
        embedding_matrix.append([0]*EMBEDDING_DIM)
    embedding_matrix = np.array(embedding_matrix)

  else:
    # only dota glossary data
    embedding_matrix = []
    for word in word_list:
        embeddings = [0] * max(domain_to_ix_glossary.values())
        if word in list(domain_to_ix_glossary.keys()):
            embeddings[domain_to_ix_glossary[word]] = 1
        embedding_matrix.append(embeddings)
    embedding_matrix = np.array(embedding_matrix)
  return embedding_matrix
  

In [ ]:
# default to set domain chat data as chat_list with len < MAX_LEN = 10
lens_restriction = True
if lens_restriction == True:
    domain_data_chat = chat_list_clean_len
else:
    domain_data_chat = chat_list_clean

# Best model Structure

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

In [ ]:
def argmax(vec):
    # from NLP lab 9
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    # from NLP lab 9
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    # from NLP lab 9
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
class MultiHeadAttention(nn.Module):
      # self-attention
      # https://github.com/ROBINADC/BiGRU-CRF-with-Attention-for-NER/blob/master/ner/gamma/attention.py
      def __init__(self, input_dim, nheads, attention_type='scaled-dot', attention_dropout = 0.2, bias = False):
          # possible attention score calculation is 
          # dot, scaled-dot, cosine, general
          super(MultiHeadAttention, self).__init__()
          self.input_dim = input_dim
          self.nheads = nheads
          self.heads_size =  input_dim // nheads
          
          self.attention_type = attention_type
          self.attention_dropout =  nn.Dropout(attention_dropout)
          self.query = nn.Linear(input_dim, input_dim, bias = bias)
          self.key = nn.Linear(input_dim, input_dim, bias = bias)
          self.value = nn.Linear(input_dim,input_dim, bias = bias)
          self.weights = None # trainable weight matrix


      def transpose_for_scores(self, x):
        # source: https://blog.csdn.net/beilizhang/article/details/115282604
        new_x_shape = x.size()[:-1] + (self.nheads, self.heads_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

      def calculate_score(self, query, key, value, mask = None):
          eps = 1e-8
          if self.attention_type == 'dot':
              scores = torch.matmul(query, key.transpose(-1, -2))
          elif self.attention_type == 'scaled-dot':
              scores = torch.matmul(query, key.transpose(-1, -2))
              # scale
              d = query.shape[-1]
              scores /= torch.sqrt(torch.tensor(d))
          elif self.attention_type == 'cosine':
              # only calculate norm for the last dimension
              q_norm = query / (query.norm(p=2, dim=-1, keepdim=True) + eps)  
              k_norm = key / (key.norm(p=2, dim=-1, keepdim=True) + eps)
              # query = (B, nheads, T_q, D)
              # key = (B, nheads, T_k, D)
              # value = (B, nheads, T_k, D) D_v: value embedding dimension
              # key permute = (B, nheads, D, T_k)
              # bmm = (B, nheads, T_q, T_k)
              scores = torch.matmul(q_norm, k_norm.permute(0, 1, 3, 2)) 
          elif self.attention_type == 'general':
              if self.weights is None:
                  self.weights = nn.Parameter(torch.empty(query.shape[-1], key.shape[-1])).to(device)
                  torch.nn.init.xavier_uniform_(self.weights) # initialize weight
              scores = query.matmul(self.weights).matmul(key.permute(0, 1, 3, 2))
          return scores 

      def forward(self, lstm_out):
          # lstm_out = (batch, seq_length ,input_size)
          query = self.query(lstm_out).to(device)
          key = self.key(lstm_out).to(device)
          value = self.key(lstm_out).to(device)


          # (batch,nheads,seq_len, head_size)
          # https://blog.csdn.net/beilizhang/article/details/115282604
          query = self.transpose_for_scores(query).to(device) 
          key = self.transpose_for_scores(key).to(device)
          value = self.transpose_for_scores(value).to(device)
          scores = self.calculate_score(query, key, value)
          attention_weights = torch.softmax(scores, dim=-1).to(device)
          attention = self.attention_dropout(attention_weights).to(device)
          # attention = (B, nheads, T_q, T_k)
          # value = (B, nheads, T_k, D)
          # output = (B, nheads, T_q, D)
          output = torch.matmul(attention, value).to(device)
          output = output.transpose(1, 2).reshape(output.shape[0], output.shape[1], -1).to(device)  # batch, n, all_heads_size
          return output


In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embeddings_dict, hidden_dim, target_size, num_layers, indicators):
        """
        Bi-Lstm layer in the model.
        :param vocab_size: dict, with word vocab and character vocab size
        :param embedding_dim: dict, with different aspects'feature's dimension
        :param embedding_dict: dict, with different aspects'feature's embedding table
                                except for character-embedding, which is randomly initialized and updated during training

        :param hidden_dim: int, the concatenated embedding dimension
        :param hidden_dim: int, the concatenated embedding dimension
        :param target_size: int, the target label size
        :param num_layers: int, the stacked layer for each bilstm block
        :param indicators: dict, to indicate whether model has specific module
        """
      
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.target_size = target_size
        self.indicators = indicators
        self.char_embs = None
        self.postag_word_embeds = None
        self.semantic_word_embeds = None
        self.domain_word_embeds = None

        emb_dim = 0
        if 'pos_tag' in list(embedding_dim.keys()):
          self.postag_word_embeds = nn.Embedding(vocab_size['word'], embedding_dim['pos_tag'])
          self.postag_word_embeds.weight.data.copy_(torch.from_numpy(embeddings_dict['pos_tag']))
          emb_dim += embedding_dim['pos_tag']

        if 'word' in list(embedding_dim.keys()):
          self.semantic_word_embeds = nn.Embedding(vocab_size['word'], embedding_dim['word'])
          self.semantic_word_embeds.weight.data.copy_(torch.from_numpy(embeddings_dict['word']))
          self.semantic_word_embeds.requires_grad = False
          emb_dim += embedding_dim['word']

        if 'char' in list(embedding_dim.keys()):
          self.char_embs = nn.Embedding(vocab_size['char'], embedding_dim['char'])
          self.char_lstm = nn.LSTM(embedding_dim['char'], embedding_dim['char']//2, 
                                   batch_first=True, bidirectional=True)
          
          nn.init.xavier_normal_(self.char_embs.weight)
          emb_dim += embedding_dim['char']

        if 'domain' in list(embedding_dim.keys()):
          self.domain_word_embeds = nn.Embedding(vocab_size['word'], embedding_dim['domain'])
          self.domain_word_embeds.weight.data.copy_(torch.from_numpy(embeddings_dict['domain']))
          emb_dim += embedding_dim['domain']
       
        self.dropout = nn.Dropout(p=0.2)
        self.layernorm = nn.LayerNorm(emb_dim)
        self.word_lstm = nn.LSTM(emb_dim, hidden_dim // 2,
                            num_layers=num_layers, batch_first = True, bidirectional=True)
        

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.target_size)

    def forward(self, sentence, char_sentence):
        # get char embedding 
        # char_sentence [[char_list],[char_list]]
        embs_list = []
        if self.char_embs is not None:
            char_emb = None
            for word in char_sentence:
                emb = self.char_embs(word)
                _, (h, _) = self.char_lstm(emb)  
                # embedding for a word
                # h:[2, embed_size= 256]
                # embed: [1, 512]
                embed = h.view(1, -1)
                # print("embed", embed.shape)
                if char_emb is None:  
                    char_emb = embed
                else:
                    char_emb = torch.cat((char_emb, embed), 0)
            # [batch, n-word, embedding_dim]    
            char_emb_sentence = char_emb.view(1, char_emb.shape[0], char_emb.shape[1])
            embs_list.append(char_emb_sentence)

        # embedding for a sentence
        # get word embeddings(aspect 1-3)
        if self.postag_word_embeds is not None:
            pos_embeds = self.postag_word_embeds(sentence).view(1, len(sentence), -1)
            embs_list.append(pos_embeds)
        if self.semantic_word_embeds is not None:
            word_embeds = self.semantic_word_embeds(sentence).view(1, len(sentence), -1)
            embs_list.append(word_embeds)
        if self.domain_word_embeds is not None:
            domain_embeds = self.domain_word_embeds(sentence).view(1, len(sentence), -1)
            embs_list.append(domain_embeds)
        # concat all embeddings
        embs = torch.cat(embs_list, 2).to(device)
        if self.indicators['is_layernorm'] == True:
            embs = self.layernorm(embs)
        if self.indicators['is_dropout'] == True:
            embs = self.dropout(embs)
        lstm_out,_ = self.word_lstm(embs) # [batch_size = 1, squence_length, hidden_size*layer_size]
        # combined = torch.cat((char_hidden, lstm_out), -1)
        if self.indicators['is_attention'] == True:
            return lstm_out.to(device)
        else:
            lstm_out = lstm_out.view(len(sentence), self.hidden_dim).to(device)
            lstm_feats = self.hidden2tag(lstm_out).to(device)
            return lstm_feats

In [ ]:
class CRF(nn.Module):
    def __init__(self,tag_to_ix):
        """
        Modified based on NLP lab 9 code.
        """
        super(CRF, self).__init__()
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))
         # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas.to(device)

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score


    def neg_log_likelihood(self, feats, tags):
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars.to(device)
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def forward(self, features):  # dont confuse this with _forward_alg above.
        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(features)
        return score, tag_seq
  

In [ ]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embeddings_dict, hidden_dim, nheads, stacked_layer, tag_to_ix, attention_type, num_layers, indicators):
        super(Model, self).__init__()
        self.bilstm = BiLSTM(vocab_size, embedding_dim, embeddings_dict, hidden_dim, len(tag_to_ix), num_layers, indicators)
        self.attention = MultiHeadAttention(hidden_dim, nheads, attention_type)
        self.crf = CRF(tag_to_ix)
        self.indicators =  indicators
        self.hidden_dim = hidden_dim
        self.hidden2tag = nn.Linear(hidden_dim, len(tag_to_ix))
        self.stacked_layer = stacked_layer
        if self.stacked_layer > 1:
            self.stacked_lstm = nn.ModuleList()
            for i in range(self.stacked_layer - 1): # removing the first layer
                self.stacked_lstm.append(nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=num_layers, batch_first = True, bidirectional=True).to(device))
                
        if indicators["is_crf"] == True:
          self.loss_func = None # if CRF is true, then loss func is None
        else:
          self.loss_func = nn.CrossEntropyLoss()

    def forward(self, sentence, words, tags):
        lstm_feats = self.bilstm(sentence, words).to(device)
        feats = lstm_feats
        
        if self.stacked_layer > 1:
          for layer in self.stacked_lstm:
            feats, _ = layer(feats)

        if self.indicators["is_attention"] == True:
            output = self.attention(feats)
            feats = output
            feats = feats.view(-1, self.hidden_dim).to(device)
            feats = self.hidden2tag(feats).to(device)
        return self.loss(feats, tags)

    def loss(self, feats, tags):
        # if no crf layer, use cross entropy loss
        if self.indicators["is_crf"] == True:
          return self.crf.neg_log_likelihood(feats, tags)
        return self.loss_func(feats,tags)

    def predict(self, sentence, words):
        lstm_feats = self.bilstm(sentence, words).to(device)
        feats = lstm_feats
        if self.stacked_layer > 1:
          for layer in self.stacked_lstm:
            feats, _ = layer(feats)

        if  self.indicators["is_attention"] == True:
            output = self.attention(feats)            
            feats = output
            feats = feats.view(-1, self.hidden_dim).to(device)
            feats = self.hidden2tag(feats).to(device)

        if  self.indicators["is_crf"] == True:
            # print(feats.shape)
            score, tag_seq = self.crf._viterbi_decode(feats)   
            return score, tag_seq

        # # else no crf layer
        # feats = feats.view(-1, self.hidden_dim).to(device)
        # feats = self.hidden2tag(feats).to(device)

        pred = feats.argmax(dim =1).tolist()
        assert(len(pred) == len(sentence))
        score = None
        return score, pred

In [ ]:
def padding(token_lst):
    new = []
    max_len = max([len(x) for x in token_lst])
    for x in token_lst:
        if len(x) <= max_len:
            x = x + [0] * (max_len - len(x))
            new.append(x)
    return new

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def cal_acc(model, input_index, input_char_index, output_index):
    with torch.no_grad():
          ground_truth = []
          predicted = []
          for i,idxs in enumerate(input_index):
              ground_truth += output_index[i]
              words = torch.tensor(padding(input_char_index[i]), dtype=torch.long).to(device)
              score, pred = model.predict(torch.tensor(idxs, dtype=torch.long).to(device), words)
              predicted += pred
          accuracy = accuracy_score(ground_truth, predicted)

    return predicted, ground_truth, accuracy


In [ ]:
def generate_test(model, test_input_index, test_input_char_index):
    test_pred = []
    model.eval()
    with torch.no_grad():
        for i, idxs in enumerate(tqdm(test_input_index)):
            sentence_words = torch.tensor(padding(test_input_char_index[i]), dtype=torch.long).to(device)
            score, pred = model.predict(torch.tensor(idxs, dtype=torch.long).to(device), sentence_words)
            output = [ix_to_tag[x] for x in pred]
            test_pred+=output
        id = [i for i in range(len(test_pred))]
        df = pd.DataFrame ({'Id': id, 'Predicted': test_pred})
        df.to_csv("prediction.csv", index = False)
    return df

In [ ]:
import datetime
from tqdm import tqdm

def bi_lstm_crf_attention(embedding_dim, embedding_dic, tag_to_ix, params, indicators, 
                          train_input_index, train_input_char_index,
                          train_output_index,
                          val_input_index, val_input_char_index,
                          test_input_index, test_input_char_index):
  """
  The function is to train and test model.
  :param embedding_dim: dict, with different aspects'feature's dimension
  :param embedding_dict: dict, with different aspects'feature's embedding table
        except for character-embedding, which is randomly initialized and updated during training.
  :param tag_to_ix: tag to index transform dictionary.
  :param params: dictionary contains parameters setting.
  :param indicators: dict, to indicate whether model has specific module
  :param train_input_index/val_input_index/test_input_index: list of list, transform word element to index
  :param train_output_index: list of list, transform tag label to index
  :param train_input_char_index/val_input_char_index/test_input_char_index: list of list, transform character element to index                              
  """
  hidden_dim = params["hidden_dim"]
  nheads = params["nheads"]
  vocab_size = params['vocab_size']
  learning_rate = params['lr']
  attention_type = params['attention_type']
  num_layers = params['num_layers']
  stacked_layer = params['stacked_layer']
  model = Model(vocab_size, embedding_dim, embedding_dic, hidden_dim, nheads, 
                stacked_layer, tag_to_ix, attention_type, num_layers, 
                indicators).to(device)
  optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  EPOCH = 2
  for epoch in range(EPOCH):  
      time1 = datetime.datetime.now()
      train_loss = 0
      model.train()
      for i, idxs in enumerate(tqdm(train_input_index)):
          tags_index = train_output_index[i]
          # Step 1. Remember that Pytorch accumulates gradients.
          # We need to clear them out before each instance
          model.zero_grad()
          # Step 2. Get our inputs ready for the network, that is,
          # turn them into Tensors of word indices.
          sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
          sentence_words = torch.tensor(padding(train_input_char_index[i]), dtype=torch.long).to(device)
          targets = torch.tensor(tags_index, dtype=torch.long).to(device)

          # Step 3. Run our forward pass.
          loss = model(sentence_in, sentence_words, targets)
          # Step 4. Compute the loss, gradients, and update the parameters by
          # calling optimizer.step()
          loss.backward()
          optimizer.step()
          train_loss+=loss.item()
      model.eval()
      # Call the cal_acc functions you implemented as required
      predicted_train, ground_truth_train, train_acc = cal_acc(model,train_input_index, train_input_char_index, train_output_index)
      predicted_val, ground_truth_val, val_acc= cal_acc(model,val_input_index, val_input_char_index, val_output_index)
      predicted_train_tag = [ix_to_tag[x] for x in predicted_train]
      truth_train_tag = [ix_to_tag[x] for x in ground_truth_train]
      predicted_val_tag = [ix_to_tag[x] for x in predicted_val]
      truth_val_tag = [ix_to_tag[x] for x in ground_truth_val]
      print("Train classification report")
      print(classification_report(truth_train_tag, predicted_train_tag, digits = 4))
      train_f1 = f1_score(truth_train_tag, predicted_train_tag, average='micro')
    
      print("Val classification report")
      print(classification_report(truth_val_tag, predicted_val_tag, digits = 4))
      val_f1 = f1_score(truth_val_tag, predicted_val_tag, average='micro')

      val_loss = 0
      for i, idxs in enumerate(tqdm(val_input_index)):
          tags_index = val_output_index[i]
          sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
          sentence_words = torch.tensor(padding(val_input_char_index[i]), dtype=torch.long).to(device)
          targets = torch.tensor(tags_index, dtype=torch.long).to(device)
          loss = model(sentence_in, sentence_words, targets)
          val_loss+=loss.item()
      time2 = datetime.datetime.now()

      print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
  
  test_predicted = generate_test(model, test_input_index, test_input_char_index)
  return val_f1, test_predicted

# Baseline Bi-LSTM CRF

In [ ]:
class BiLSTM_CRF(nn.Module):
    """
    Baseline code from Lab 9
    """
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

    def predict(self, sentence):
        lstm_feats = self._get_lstm_features(sentence)
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
# baseline model setting from lab 9
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    time1 = datetime.datetime.now() 
    train_loss = 0
    for i, idxs in enumerate(tqdm(train_input_index)):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long)
        targets = torch.tensor(train_output_index[i], dtype=torch.long)
        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
  
    val_loss = 0
    predicted_val = []
    true_val = []
    for i, idxs in enumerate(tqdm(val_input_index)):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long)
        # sentence_words = torch.tensor(padding(val_input_char_index[i]), dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        _, pred = model.predict(sentence_in)
        predicted_val+=pred
        true_val += tags_index
        val_loss+=loss.item()
    predicted_val_tag = [ix_to_tag[x] for x in predicted_val]
    truth_val_tag = [ix_to_tag[x] for x in true_val]
    print("Val classification report")
    print(classification_report(truth_val_tag, predicted_val_tag, digits = 4))
    val_f1 = f1_score(truth_val_tag, predicted_val_tag, average='micro')
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss, val_loss, val_f1, (time2-time1).total_seconds()))
  

100%|██████████| 8705/8705 [01:12<00:00, 119.84it/s]


Val classification report
              precision    recall  f1-score   support

           C     0.7963    0.0262    0.0507      1641
           D     0.0000    0.0000    0.0000       398
           O     0.8134    0.9818    0.8897     18985
           P     0.9356    0.8897    0.9121      3936
           S     0.9109    0.7568    0.8267      3322
        SEPA     0.9928    0.9994    0.9961      3603
           T     0.6211    0.1082    0.1843      1469

    accuracy                         0.8532     33354
   macro avg     0.7243    0.5375    0.5514     33354
weighted avg     0.8379    0.8532    0.8146     33354

Epoch:1, Training loss: 74812.74, val loss: 17817.07, val f1: 0.8532, time: 384.71s


100%|██████████| 8705/8705 [01:12<00:00, 119.39it/s]


Val classification report
              precision    recall  f1-score   support

           C     0.8870    0.1578    0.2680      1641
           D     0.0000    0.0000    0.0000       398
           O     0.8583    0.9847    0.9171     18985
           P     0.9540    0.9177    0.9355      3936
           S     0.9277    0.8233    0.8724      3322
        SEPA     0.9997    1.0000    0.9999      3603
           T     0.6847    0.4391    0.5350      1469

    accuracy                         0.8859     33354
   macro avg     0.7588    0.6175    0.6468     33354
weighted avg     0.8753    0.8859    0.8641     33354

Epoch:2, Training loss: 47370.45, val loss: 13844.47, val f1: 0.8859, time: 389.55s


# Ablation Study

## Ablation Study - Embeddings

In [ ]:
# when testing embeddings, fixed other parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 1,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}


### Syntactic Embedding

#### POS tagging embedding

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
embedding_dim_pos = {'pos_tag': postag_emb.shape[1]}
embedding_dic_pos = {'pos_tag': postag_emb}
emb_pos_tagging_f1, emb_pos_tagging_test = bi_lstm_crf_attention(embedding_dim_pos, embedding_dic_pos, tag_to_ix, params, 
                                                                 indicators, 
                                                                 train_input_index, train_input_char_index,
                                                                 train_output_index,
                                                                 val_input_index, val_input_char_index,
                                                                 test_input_index, test_input_char_index)
emb_pos_tagging_f1

100%|██████████| 26078/26078 [11:34<00:00, 37.53it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9624    0.9912    0.9766      4780
           D     0.9207    0.8297    0.8728      1274
           O     0.9978    0.9978    0.9978     56815
           P     0.9992    0.9985    0.9989     11997
           S     0.9947    0.9941    0.9944     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9751    0.9753    0.9752      4292

    accuracy                         0.9943     99611
   macro avg     0.9786    0.9695    0.9737     99611
weighted avg     0.9942    0.9943    0.9942     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9619    0.9695    0.9657      1641
           D     0.9788    0.8116    0.8874       398
           O     0.9919    0.9986    0.9953     18985
           P     0.9987    0.9985    0.9986      3936
           S     0.9939    0.9874    0.9906      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:36<00:00, 89.91it/s] 


Epoch:1, Training loss: 24591.63, train acc: 0.9943, train f1: 0.9943, val loss: 1515.06, val acc: 0.9919, val f1: 0.9919, time: 940.27s


100%|██████████| 26078/26078 [11:40<00:00, 37.23it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9964    0.9941    0.9953      4780
           D     0.9120    0.9843    0.9468      1274
           O     0.9993    0.9992    0.9993     56815
           P     0.9987    0.9987    0.9987     11997
           S     0.9994    0.9966    0.9980     10035
        SEPA     1.0000    0.9999    1.0000     10418
           T     0.9939    0.9804    0.9871      4292

    accuracy                         0.9977     99611
   macro avg     0.9857    0.9933    0.9893     99611
weighted avg     0.9978    0.9977    0.9978     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9981    0.9683    0.9830      1641
           D     0.9074    0.9598    0.9328       398
           O     0.9930    0.9996    0.9963     18985
           P     0.9992    0.9977    0.9985      3936
           S     0.9994    0.9895    0.9944      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:41<00:00, 85.87it/s] 


Epoch:2, Training loss: 3052.37, train acc: 0.9977, train f1: 0.9977, val loss: 1513.33, val acc: 0.9943, val f1: 0.9943, time: 954.83s


100%|██████████| 500/500 [00:02<00:00, 189.41it/s]


0.9943035318102776

### Semantic Embedding

#### glove-twitter-50

In [ ]:
word_emb = word_embedding_table(word_list, pretrained_50 = True)
embedding_dim_pretrained_50 = {'word': word_emb.shape[1]}
embedding_dic_pretrained_50 = {'word': word_emb}
emb_pos_pretrained_50_f1, emb_pos_pretrained_50_test = bi_lstm_crf_attention(embedding_dim_pretrained_50, embedding_dic_pretrained_50, tag_to_ix, params, indicators,
                                                                             train_input_index, train_input_char_index,
                                                                              train_output_index,
                                                                              val_input_index, val_input_char_index,
                                                                              test_input_index, test_input_char_index)
emb_pos_pretrained_50_f1

[==================================================] 100.0% 199.5/199.5MB downloaded


100%|██████████| 26078/26078 [20:24<00:00, 21.30it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9498    0.9613    0.9555      4780
           D     0.9270    0.7072    0.8023      1274
           O     0.9852    0.9942    0.9897     56815
           P     0.9966    0.9939    0.9952     11997
           S     0.9848    0.9763    0.9805     10035
        SEPA     0.9991    1.0000    0.9996     10418
           T     0.9884    0.9499    0.9688      4292

    accuracy                         0.9858     99611
   macro avg     0.9758    0.9404    0.9559     99611
weighted avg     0.9857    0.9858    0.9855     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9520    0.9433    0.9477      1641
           D     0.9204    0.6683    0.7744       398
           O     0.9817    0.9959    0.9887     18985
           P     0.9982    0.9916    0.9949      3936
           S     0.9899    0.9693    0.9795      3322
        SEPA     0.9994 

100%|██████████| 8705/8705 [02:06<00:00, 68.55it/s]


Epoch:1, Training loss: 33569.72, train acc: 0.9858, train f1: 0.9858, val loss: 2079.48, val acc: 0.9846, val f1: 0.9846, time: 1583.60s


...

Train classification report
              precision    recall  f1-score   support

           C     0.9841    0.9822    0.9831      4780
           D     0.9638    0.8995    0.9306      1274
           O     0.9952    0.9983    0.9967     56815
           P     0.9991    0.9980    0.9985     11997
           S     0.9936    0.9921    0.9929     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9932    0.9797    0.9864      4292

    accuracy                         0.9950     99611
   macro avg     0.9898    0.9786    0.9840     99611
weighted avg     0.9949    0.9950    0.9949     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9814    0.9622    0.9717      1641
           D     0.9584    0.8693    0.9117       398
           O     0.9901    0.9974    0.9937     18985
           P     0.9987    0.9975    0.9981      3936
           S     0.9933    0.9846    0.9890      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:07<00:00, 68.47it/s]


Epoch:2, Training loss: 5380.50, train acc: 0.9950, train f1: 0.9950, val loss: 1543.22, val acc: 0.9918, val f1: 0.9918, time: 1586.91s


100%|██████████| 500/500 [00:04<00:00, 114.29it/s]


0.9918150746537147

#### glove-twitter-100

In [ ]:
word_emb = word_embedding_table(word_list, pretrained_100 = True)
embedding_dim_pretrained_100 = {'word': word_emb.shape[1]}
embedding_dic_pretrained_100 = {'word': word_emb}
emb_pos_pretrained_100_f1, emb_pos_pretrained_100_test = bi_lstm_crf_attention(embedding_dim_pretrained_100, embedding_dic_pretrained_100, tag_to_ix, params, indicators,
                                                                             train_input_index, train_input_char_index,
                                                                              train_output_index,
                                                                              val_input_index, val_input_char_index,
                                                                              test_input_index, test_input_char_index)
emb_pos_pretrained_100_f1

[==================================================] 100.0% 387.1/387.1MB downloaded


100%|██████████| 26078/26078 [21:18<00:00, 20.40it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9417    0.9764    0.9587      4780
           D     0.8823    0.8061    0.8425      1274
           O     0.9896    0.9941    0.9918     56815
           P     0.9987    0.9960    0.9974     11997
           S     0.9933    0.9801    0.9867     10035
        SEPA     0.9992    1.0000    0.9996     10418
           T     0.9906    0.9529    0.9714      4292

    accuracy                         0.9885     99611
   macro avg     0.9708    0.9579    0.9640     99611
weighted avg     0.9885    0.9885    0.9884     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9421    0.9622    0.9521      1641
           D     0.8902    0.7739    0.8280       398
           O     0.9870    0.9951    0.9910     18985
           P     0.9990    0.9975    0.9982      3936
           S     0.9957    0.9729    0.9842      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:09<00:00, 67.42it/s]


Epoch:1, Training loss: 30892.01, train acc: 0.9885, train f1: 0.9885, val loss: 1841.51, val acc: 0.9876, val f1: 0.9876, time: 1645.49s


...

Train classification report
              precision    recall  f1-score   support

           C     0.9874    0.9870    0.9872      4780
           D     0.9622    0.8980    0.9289      1274
           O     0.9956    0.9980    0.9968     56815
           P     0.9987    0.9987    0.9987     11997
           S     0.9933    0.9939    0.9936     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9946    0.9807    0.9876      4292

    accuracy                         0.9954     99611
   macro avg     0.9902    0.9795    0.9847     99611
weighted avg     0.9953    0.9954    0.9953     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9882    0.9683    0.9781      1641
           D     0.9410    0.8819    0.9105       398
           O     0.9916    0.9972    0.9944     18985
           P     0.9987    0.9990    0.9989      3936
           S     0.9918    0.9886    0.9902      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:06<00:00, 68.74it/s]


Epoch:2, Training loss: 5084.68, train acc: 0.9954, train f1: 0.9954, val loss: 1481.87, val acc: 0.9927, val f1: 0.9927, time: 1631.48s


100%|██████████| 500/500 [00:03<00:00, 132.42it/s]


0.9926545541764106

#### word2vec (cbow)

In [ ]:
word_emb = word_embedding_table(word_list, word2vec_cbow = True)
embedding_dim_w2v_cbow = {'word': word_emb.shape[1]}
embedding_dic_w2v_cbow = {'word': word_emb}

emb_pos_w2v_cbow_f1, emb_pos_w2v_cbow_test = bi_lstm_crf_attention(embedding_dim_w2v_cbow, embedding_dic_w2v_cbow, tag_to_ix, params, indicators, 
                                                                   train_input_index, train_input_char_index,
                                                                   train_output_index,
                                                                   val_input_index, val_input_char_index,
                                                                   test_input_index, test_input_char_index)
emb_pos_w2v_cbow_f1

100%|██████████| 26078/26078 [21:10<00:00, 20.52it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9194    0.8713    0.8947      4780
           D     0.5892    0.4301    0.4973      1274
           O     0.9746    0.9894    0.9820     56815
           P     0.9899    0.9840    0.9869     11997
           S     0.9747    0.9466    0.9604     10035
        SEPA     0.9859    0.9934    0.9896     10418
           T     0.8261    0.8290    0.8275      4292

    accuracy                         0.9651     99611
   macro avg     0.8943    0.8634    0.8769     99611
weighted avg     0.9637    0.9651    0.9641     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9375    0.8678    0.9013      1641
           D     0.6955    0.3844    0.4951       398
           O     0.9711    0.9966    0.9837     18985
           P     0.9969    0.9850    0.9909      3936
           S     0.9859    0.9464    0.9658      3322
        SEPA     0.9895 

100%|██████████| 8705/8705 [02:06<00:00, 68.65it/s]


Epoch:1, Training loss: 43674.89, train acc: 0.9651, train f1: 0.9651, val loss: 4019.96, val acc: 0.9688, val f1: 0.9688, time: 1632.53s


100%|██████████| 26078/26078 [21:05<00:00, 20.60it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9740    0.9812    0.9776      4780
           D     0.8517    0.8838    0.8675      1274
           O     0.9943    0.9942    0.9943     56815
           P     0.9984    0.9926    0.9955     11997
           S     0.9878    0.9882    0.9880     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9617    0.9599    0.9608      4292

    accuracy                         0.9905     99611
   macro avg     0.9669    0.9714    0.9691     99611
weighted avg     0.9906    0.9905    0.9905     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9844    0.9634    0.9738      1641
           D     0.9079    0.8920    0.8999       398
           O     0.9897    0.9976    0.9936     18985
           P     0.9990    0.9929    0.9959      3936
           S     0.9927    0.9825    0.9876      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:07<00:00, 68.22it/s]


Epoch:2, Training loss: 8496.20, train acc: 0.9905, train f1: 0.9905, val loss: 1809.40, val acc: 0.9904, val f1: 0.9904, time: 1627.27s


100%|██████████| 500/500 [00:03<00:00, 127.65it/s]


0.9903759669005217

#### word2vec (skip-gram)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

word_emb = word_embedding_table(word_list, word2vec_sg = True)
embedding_dim_w2v_sg = {'word': word_emb.shape[1]}
embedding_dic_w2v_sg = {'word': word_emb}

emb_pos_w2v_sg_f1, emb_pos_w2v_sg_test = bi_lstm_crf_attention(embedding_dim_w2v_sg, embedding_dic_w2v_sg, tag_to_ix, params, indicators,
                                                               train_input_index, train_input_char_index,
                                                                train_output_index,
                                                                val_input_index, val_input_char_index,
                                                                test_input_index, test_input_char_index)
emb_pos_w2v_sg_f1

100%|██████████| 26078/26078 [21:09<00:00, 20.54it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9024    0.9613    0.9309      4780
           D     0.7560    0.6641    0.7071      1274
           O     0.9866    0.9901    0.9884     56815
           P     0.9964    0.9578    0.9767     11997
           S     0.9751    0.9870    0.9810     10035
        SEPA     0.9991    1.0000    0.9996     10418
           T     0.9716    0.9658    0.9687      4292

    accuracy                         0.9804     99611
   macro avg     0.9410    0.9323    0.9361     99611
weighted avg     0.9803    0.9804    0.9802     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9230    0.9427    0.9328      1641
           D     0.8942    0.6583    0.7583       398
           O     0.9795    0.9972    0.9883     18985
           P     0.9987    0.9510    0.9742      3936
           S     0.9903    0.9822    0.9862      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:10<00:00, 66.90it/s]


Epoch:1, Training loss: 28514.58, train acc: 0.9804, train f1: 0.9804, val loss: 2458.08, val acc: 0.9816, val f1: 0.9816, time: 1637.95s


100%|██████████| 26078/26078 [21:01<00:00, 20.68it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9782    0.9843    0.9812      4780
           D     0.8943    0.9560    0.9241      1274
           O     0.9979    0.9964    0.9972     56815
           P     0.9977    0.9983    0.9980     11997
           S     0.9935    0.9950    0.9943     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9929    0.9790    0.9859      4292

    accuracy                         0.9951     99611
   macro avg     0.9792    0.9870    0.9830     99611
weighted avg     0.9951    0.9951    0.9951     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9869    0.9653    0.9760      1641
           D     0.9566    0.9422    0.9494       398
           O     0.9919    0.9990    0.9955     18985
           P     0.9997    0.9977    0.9987      3936
           S     0.9961    0.9898    0.9929      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:08<00:00, 67.95it/s]


Epoch:2, Training loss: 4831.09, train acc: 0.9951, train f1: 0.9951, val loss: 1608.45, val acc: 0.9938, val f1: 0.9938, time: 1625.99s


100%|██████████| 500/500 [00:03<00:00, 129.63it/s]


0.9937638664028302

#### FastText (cbow)

In [ ]:
word_emb = word_embedding_table(word_list, fasttext_cbow = True)
embedding_dim_fast_cbow = {'word': word_emb.shape[1]}
embedding_dic_fast_cbow = {'word': word_emb}

emb_pos_fast_cbow_f1, emb_pos_fast_cbow_test = bi_lstm_crf_attention(embedding_dim_fast_cbow, embedding_dic_fast_cbow, tag_to_ix, params, indicators,
                                                                    train_input_index, train_input_char_index,
                                                                    train_output_index,
                                                                    val_input_index, val_input_char_index,
                                                                    test_input_index, test_input_char_index)
emb_pos_fast_cbow_f1

100%|██████████| 26078/26078 [21:42<00:00, 20.02it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.8570    0.9467    0.8996      4780
           D     0.7960    0.3705    0.5056      1274
           O     0.9901    0.9983    0.9942     56815
           P     0.9968    0.9932    0.9950     11997
           S     0.9918    0.9820    0.9869     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9470    0.9152    0.9308      4292

    accuracy                         0.9821     99611
   macro avg     0.9398    0.8865    0.9017     99611
weighted avg     0.9814    0.9821    0.9806     99611

Val classification report
              precision    recall  f1-score   support

           C     0.8923    0.9238    0.9078      1641
           D     0.7670    0.3970    0.5232       398
           O     0.9860    0.9991    0.9925     18985
           P     0.9977    0.9931    0.9954      3936
           S     0.9933    0.9771    0.9851      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:08<00:00, 67.53it/s]


Epoch:1, Training loss: 44543.27, train acc: 0.9821, train f1: 0.9821, val loss: 2659.19, val acc: 0.9816, val f1: 0.9816, time: 1668.52s


100%|██████████| 26078/26078 [22:25<00:00, 19.38it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9689    0.9847    0.9768      4780
           D     0.8799    0.8917    0.8858      1274
           O     0.9974    0.9989    0.9982     56815
           P     0.9995    0.9938    0.9967     11997
           S     0.9967    0.9920    0.9944     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9834    0.9683    0.9758      4292

    accuracy                         0.9943     99611
   macro avg     0.9751    0.9756    0.9754     99611
weighted avg     0.9944    0.9943    0.9944     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9729    0.9634    0.9682      1641
           D     0.9034    0.8693    0.8860       398
           O     0.9914    0.9995    0.9954     18985
           P     1.0000    0.9939    0.9969      3936
           S     0.9982    0.9868    0.9924      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:23<00:00, 60.46it/s]


Epoch:2, Training loss: 6426.49, train acc: 0.9943, train f1: 0.9943, val loss: 1857.91, val acc: 0.9920, val f1: 0.9920, time: 1726.26s


100%|██████████| 500/500 [00:04<00:00, 116.21it/s]


0.9919949631228638

#### FastText (skip-gram)

In [ ]:
word_emb = word_embedding_table(word_list, fasttext_sg = True)
embedding_dim_fast_sg = {'word': word_emb.shape[1]}
embedding_dic_fast_sg = {'word': word_emb}
emb_pos_fast_sg_f1, emb_pos_fast_sg_test = bi_lstm_crf_attention(embedding_dim_fast_sg, embedding_dic_fast_sg, tag_to_ix, params, indicators,
                                                                  train_input_index, train_input_char_index,
                                                                  train_output_index,
                                                                  val_input_index, val_input_char_index,
                                                                  test_input_index, test_input_char_index)
emb_pos_fast_sg_f1

100%|██████████| 26078/26078 [22:02<00:00, 19.72it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9631    0.9933    0.9780      4780
           D     0.8992    0.8336    0.8652      1274
           O     0.9970    0.9981    0.9975     56815
           P     0.9990    0.9970    0.9980     11997
           S     0.9968    0.9934    0.9951     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9806    0.9651    0.9728      4292

    accuracy                         0.9940     99611
   macro avg     0.9765    0.9686    0.9723     99611
weighted avg     0.9939    0.9940    0.9939     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9719    0.9701    0.9710      1641
           D     0.9153    0.8141    0.8617       398
           O     0.9913    0.9982    0.9947     18985
           P     0.9995    0.9947    0.9971      3936
           S     0.9967    0.9901    0.9934      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:06<00:00, 68.60it/s]


Epoch:1, Training loss: 31756.42, train acc: 0.9940, train f1: 0.9940, val loss: 1388.30, val acc: 0.9913, val f1: 0.9913, time: 1683.64s


100%|██████████| 26078/26078 [21:10<00:00, 20.52it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9892    0.9977    0.9934      4780
           D     0.9814    0.9513    0.9661      1274
           O     0.9989    0.9989    0.9989     56815
           P     0.9964    0.9993    0.9979     11997
           S     0.9987    0.9971    0.9979     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9946    0.9897    0.9922      4292

    accuracy                         0.9978     99611
   macro avg     0.9942    0.9906    0.9923     99611
weighted avg     0.9978    0.9978    0.9978     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9925    0.9738    0.9831      1641
           D     0.9945    0.9146    0.9529       398
           O     0.9929    0.9996    0.9962     18985
           P     0.9975    0.9987    0.9981      3936
           S     0.9991    0.9910    0.9950      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:05<00:00, 69.39it/s]


Epoch:2, Training loss: 3341.89, train acc: 0.9978, train f1: 0.9978, val loss: 1585.32, val acc: 0.9948, val f1: 0.9948, time: 1623.91s


100%|██████████| 500/500 [00:03<00:00, 128.93it/s]


0.994843197217725

#### Character embedding

In [ ]:
embedding_dim_word_char = {'char': params["char_emb"]}
embedding_dic_word_char = {'pos_tag': None, 'word': None, 'domain':None}
emb_word_char_f1, emb_word_char_test = bi_lstm_crf_attention(embedding_dim_word_char, embedding_dic_word_char, tag_to_ix, params, indicators,
                                                            train_input_index, train_input_char_index,
                                                            train_output_index,
                                                            val_input_index, val_input_char_index,
                                                            test_input_index, test_input_char_index)
emb_word_char_f1

100%|██████████| 26078/26078 [31:04<00:00, 13.98it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.7446    0.2538    0.3785      4780
           D     0.0000    0.0000    0.0000      1274
           O     0.8398    0.9894    0.9085     56815
           P     0.9835    0.8459    0.9095     11997
           S     0.9177    0.7818    0.8443     10035
        SEPA     0.9997    1.0000    0.9999     10418
           T     0.9732    0.3977    0.5647      4292

    accuracy                         0.8789     99611
   macro avg     0.7798    0.6098    0.6579     99611
weighted avg     0.8721    0.8789    0.8598     99611

Val classification report
              precision    recall  f1-score   support

           C     0.7557    0.2413    0.3658      1641
           D     0.0000    0.0000    0.0000       398
           O     0.8395    0.9905    0.9087     18985
           P     0.9875    0.8458    0.9112      3936
           S     0.9153    0.7875    0.8466      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:41<00:00, 53.77it/s]


Epoch:1, Training loss: 72053.21, train acc: 0.8789, train f1: 0.8789, val loss: 14170.31, val acc: 0.8794, val f1: 0.8794, time: 2357.93s


100%|██████████| 26078/26078 [29:29<00:00, 14.74it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.8993    0.8203    0.8580      4780
           D     0.9368    0.1978    0.3266      1274
           O     0.9411    0.9755    0.9580     56815
           P     0.9682    0.9910    0.9795     11997
           S     0.9779    0.8405    0.9040     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.7937    0.8814    0.8353      4292

    accuracy                         0.9449     99611
   macro avg     0.9310    0.8152    0.8373     99611
weighted avg     0.9458    0.9449    0.9414     99611

Val classification report
              precision    recall  f1-score   support

           C     0.8785    0.8020    0.8385      1641
           D     0.8947    0.1709    0.2869       398
           O     0.9393    0.9716    0.9552     18985
           P     0.9648    0.9886    0.9765      3936
           S     0.9769    0.8399    0.9032      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:27<00:00, 59.06it/s]


Epoch:2, Training loss: 28236.71, train acc: 0.9449, train f1: 0.9449, val loss: 7218.20, val acc: 0.9418, val f1: 0.9418, time: 2221.01s


100%|██████████| 500/500 [00:05<00:00, 96.86it/s]


0.9418360616417821

### Domain Embedding

#### External dota chat

In [ ]:
# default set length restriction, let the glossary word length smaller than 10
# remove word like ahhhhhhhhhhhhhhh
lens_restriction = True
if lens_restriction == True:
    domain_data_chat = chat_list_clean_len
else:
    domain_data_chat = chat_list_clean

domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = True)
embedding_dim_domain_ex = {'domain': domain_emb.shape[1]}
embedding_dic_domain_ex = {'domain': domain_emb}

emb_domain_ex_f1, emb_pos_domain_ex_test = bi_lstm_crf_attention(embedding_dim_domain_ex, embedding_dic_domain_ex, tag_to_ix, params, indicators,
                                                                 train_input_index, train_input_char_index,
                                                                  train_output_index,
                                                                  val_input_index, val_input_char_index,
                                                                  test_input_index, test_input_char_index)
emb_domain_ex_f1

100%|██████████| 26078/26078 [21:20<00:00, 20.36it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9800    0.9962    0.9881      4780
           D     0.9204    0.9262    0.9233      1274
           O     0.9941    0.9961    0.9951     56815
           P     0.9992    0.9979    0.9985     11997
           S     0.9984    0.9890    0.9937     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9812    0.9595    0.9702      4292

    accuracy                         0.9935     99611
   macro avg     0.9819    0.9807    0.9813     99611
weighted avg     0.9936    0.9935    0.9935     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9636    0.9854    0.9744      1641
           D     0.9111    0.9271    0.9191       398
           O     0.9916    0.9935    0.9926     18985
           P     0.9990    0.9972    0.9981      3936
           S     0.9976    0.9859    0.9917      3322
        SEPA     0.9992 

100%|██████████| 8705/8705 [02:26<00:00, 59.31it/s]


Epoch:1, Training loss: 24453.15, train acc: 0.9935, train f1: 0.9935, val loss: 1363.08, val acc: 0.9909, val f1: 0.9909, time: 1676.07s


100%|██████████| 26078/26078 [22:50<00:00, 19.02it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9996    0.9941    0.9969      4780
           D     0.9857    0.9741    0.9799      1274
           O     0.9925    0.9998    0.9961     56815
           P     1.0000    0.9762    0.9879     11997
           S     0.9997    0.9960    0.9979     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9958    0.9830    0.9893      4292

    accuracy                         0.9953     99611
   macro avg     0.9962    0.9890    0.9926     99611
weighted avg     0.9953    0.9953    0.9952     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9981    0.9695    0.9836      1641
           D     0.9769    0.9548    0.9657       398
           O     0.9873    0.9993    0.9932     18985
           P     1.0000    0.9769    0.9883      3936
           S     0.9991    0.9904    0.9947      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:10<00:00, 66.76it/s]


Epoch:2, Training loss: 3027.72, train acc: 0.9953, train f1: 0.9953, val loss: 1505.89, val acc: 0.9920, val f1: 0.9920, time: 1737.66s


100%|██████████| 500/500 [00:04<00:00, 120.41it/s]


0.9920249445343887

#### dota-specific terms

In [ ]:
domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = False)
embedding_dim_domain = {'domain': domain_emb.shape[1]}
embedding_dic_domain = {'domain': domain_emb}

emb_domain_f1, emb_domain_test = bi_lstm_crf_attention(embedding_dim_domain, embedding_dic_domain, tag_to_ix, params, indicators,
                                                       train_input_index, train_input_char_index, train_output_index,
                                                       val_input_index, val_input_char_index,
                                                       test_input_index, test_input_char_index)
emb_domain_f1

100%|██████████| 26078/26078 [21:20<00:00, 20.37it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9603    0.9927    0.9762      4780
           D     0.9100    0.9364    0.9230      1274
           O     0.9966    0.9927    0.9947     56815
           P     0.9987    0.9969    0.9978     11997
           S     0.9961    0.9828    0.9894     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9485    0.9874    0.9676      4292

    accuracy                         0.9920     99611
   macro avg     0.9729    0.9841    0.9784     99611
weighted avg     0.9922    0.9920    0.9921     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9724    0.9671    0.9698      1641
           D     0.9430    0.9146    0.9286       398
           O     0.9896    0.9958    0.9927     18985
           P     0.9982    0.9949    0.9966      3936
           S     0.9988    0.9765    0.9875      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:09<00:00, 67.03it/s]


Epoch:1, Training loss: 20806.14, train acc: 0.9920, train f1: 0.9920, val loss: 1449.90, val acc: 0.9901, val f1: 0.9901, time: 1664.64s


100%|██████████| 26078/26078 [21:50<00:00, 19.90it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9923    0.9956    0.9939      4780
           D     0.9565    0.9498    0.9531      1274
           O     0.9985    0.9971    0.9978     56815
           P     0.9926    0.9990    0.9958     11997
           S     0.9969    0.9963    0.9966     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9802    0.9809    0.9806      4292

    accuracy                         0.9962     99611
   macro avg     0.9882    0.9884    0.9883     99611
weighted avg     0.9962    0.9962    0.9962     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9931    0.9714    0.9821      1641
           D     0.9812    0.9171    0.9481       398
           O     0.9920    0.9991    0.9955     18985
           P     0.9967    0.9964    0.9966      3936
           S     0.9991    0.9913    0.9952      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:10<00:00, 66.49it/s]


Epoch:2, Training loss: 3437.09, train acc: 0.9962, train f1: 0.9962, val loss: 1394.09, val acc: 0.9939, val f1: 0.9939, time: 1702.99s


100%|██████████| 500/500 [00:04<00:00, 116.73it/s]


0.9939137734604545

#### external dota chat+dota-specific terms

In [ ]:
domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = False, concate = True)
embedding_dim_domain_concate = {'domain': domain_emb.shape[1]}
embedding_dic_domain_concate = {'domain': domain_emb}

emb_domain_f1_concate, emb_domain_test_concate = bi_lstm_crf_attention(embedding_dim_domain_concate, embedding_dic_domain_concate, tag_to_ix, params, indicators,
                                                                       train_input_index, train_input_char_index, train_output_index,
                                                                       val_input_index, val_input_char_index,
                                                                       test_input_index, test_input_char_index)
emb_domain_f1_concate

100%|██████████| 26078/26078 [21:30<00:00, 20.21it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9930    0.9766    0.9847      4780
           D     0.9303    0.9325    0.9314      1274
           O     0.9949    0.9953    0.9951     56815
           P     0.9988    0.9904    0.9946     11997
           S     0.9940    0.9905    0.9923     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9453    0.9867    0.9656      4292

    accuracy                         0.9927     99611
   macro avg     0.9795    0.9817    0.9805     99611
weighted avg     0.9928    0.9927    0.9927     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9937    0.9549    0.9739      1641
           D     0.9562    0.9322    0.9440       398
           O     0.9895    0.9975    0.9935     18985
           P     0.9992    0.9898    0.9945      3936
           S     0.9963    0.9852    0.9908      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:11<00:00, 66.21it/s]


Epoch:1, Training loss: 19349.20, train acc: 0.9927, train f1: 0.9927, val loss: 1366.41, val acc: 0.9911, val f1: 0.9911, time: 1666.47s


100%|██████████| 26078/26078 [21:08<00:00, 20.56it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9931    0.9948    0.9939      4780
           D     0.9690    0.9074    0.9372      1274
           O     0.9982    0.9976    0.9979     56815
           P     0.9990    0.9979    0.9985     11997
           S     0.9958    0.9938    0.9948     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9553    0.9865    0.9707      4292

    accuracy                         0.9958     99611
   macro avg     0.9872    0.9826    0.9847     99611
weighted avg     0.9958    0.9958    0.9958     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9938    0.9732    0.9834      1641
           D     0.9529    0.8643    0.9065       398
           O     0.9923    0.9983    0.9953     18985
           P     0.9997    0.9952    0.9975      3936
           S     0.9964    0.9886    0.9924      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:09<00:00, 67.21it/s]


Epoch:2, Training loss: 4803.37, train acc: 0.9958, train f1: 0.9958, val loss: 1786.48, val acc: 0.9925, val f1: 0.9925, time: 1634.67s


100%|██████████| 500/500 [00:03<00:00, 126.94it/s]


0.9925046471187864

### Embedding Combinations

#### aspects 1+2+3
#### Pos Embedding + FastText SG + Domain Embedding

In [ ]:
lens_restriction = True
if lens_restriction == True:
    domain_data_chat = chat_list_clean_len
else:
    domain_data_chat = chat_list_clean

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
word_emb_fast_sg = word_embedding_table(word_list, fasttext_sg = True)

domain_emb_ex = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = True)
embedding_dim_concate = {'pos_tag': postag_emb.shape[1], 'word': word_emb_fast_sg.shape[1], 'domain': domain_emb_ex.shape[1]}
embedding_dic_concate = {'pos_tag': postag_emb, 'word': word_emb_fast_sg, 'domain': domain_emb_ex}
emb_domain_f1_concate, emb_domain_test_concate = bi_lstm_crf_attention(embedding_dim_concate, embedding_dic_concate, tag_to_ix, params, indicators,
                                                                       train_input_index, train_input_char_index, train_output_index,
                                                                       val_input_index, val_input_char_index,
                                                                       test_input_index, test_input_char_index)
emb_domain_f1_concate

100%|██████████| 26078/26078 [12:07<00:00, 35.84it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9929    0.9921    0.9925      4780
           D     0.9744    0.9270    0.9501      1274
           O     0.9920    0.9993    0.9956     56815
           P     0.9997    0.9976    0.9986     11997
           S     0.9991    0.9865    0.9928     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9985    0.9513    0.9743      4292

    accuracy                         0.9945     99611
   macro avg     0.9938    0.9791    0.9863     99611
weighted avg     0.9946    0.9945    0.9945     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9806    0.9842    0.9824      1641
           D     0.9656    0.9171    0.9407       398
           O     0.9891    0.9978    0.9934     18985
           P     0.9990    0.9972    0.9981      3936
           S     0.9985    0.9819    0.9901      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:43<00:00, 83.78it/s]


Epoch:1, Training loss: 22973.22, train acc: 0.9945, train f1: 0.9945, val loss: 1300.40, val acc: 0.9921, val f1: 0.9921, time: 988.36s


100%|██████████| 26078/26078 [11:52<00:00, 36.62it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9973    0.9977    0.9975      4780
           D     0.9976    0.9694    0.9833      1274
           O     0.9983    0.9997    0.9990     56815
           P     0.9999    0.9995    0.9997     11997
           S     0.9983    0.9968    0.9976     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9951    0.9888    0.9919      4292

    accuracy                         0.9985     99611
   macro avg     0.9981    0.9931    0.9956     99611
weighted avg     0.9985    0.9985    0.9985     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9908    0.9842    0.9875      1641
           D     0.9844    0.9497    0.9668       398
           O     0.9939    0.9978    0.9958     18985
           P     0.9992    0.9995    0.9994      3936
           S     0.9958    0.9928    0.9943      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:43<00:00, 84.03it/s]


Epoch:2, Training loss: 2847.82, train acc: 0.9985, train f1: 0.9985, val loss: 961.04, val acc: 0.9950, val f1: 0.9950, time: 967.84s


100%|██████████| 500/500 [00:02<00:00, 186.34it/s]


0.9949631228638244

#### aspects 1+2
#### Pos Embedding + FastText SG

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
word_emb_fast_sg = word_embedding_table(word_list, fasttext_sg = True)
embedding_dim_concate12 = {'pos_tag': postag_emb.shape[1], 'word': word_emb_fast_sg.shape[1]}
embedding_dic_concate12 = {'pos_tag': postag_emb, 'word': word_emb_fast_sg}
emb_domain_f1_concate12, emb_domain_test_concate12 = bi_lstm_crf_attention(embedding_dim_concate12, embedding_dic_concate12, tag_to_ix, params, indicators,
                                                                           train_input_index, train_input_char_index, train_output_index,
                                                                           val_input_index, val_input_char_index,
                                                                           test_input_index, test_input_char_index)
emb_domain_f1_concate12

100%|██████████| 26078/26078 [12:02<00:00, 36.08it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9767    0.9805    0.9786      4780
           D     0.8391    0.9537    0.8927      1274
           O     0.9911    0.9977    0.9944     56815
           P     0.9994    0.9700    0.9845     11997
           S     0.9951    0.9870    0.9910     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9921    0.9602    0.9758      4292

    accuracy                         0.9905     99611
   macro avg     0.9705    0.9784    0.9739     99611
weighted avg     0.9908    0.9905    0.9906     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9758    0.9592    0.9674      1641
           D     0.8635    0.9221    0.8919       398
           O     0.9854    0.9988    0.9921     18985
           P     0.9992    0.9693    0.9840      3936
           S     0.9969    0.9798    0.9883      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:42<00:00, 85.19it/s]


Epoch:1, Training loss: 30212.67, train acc: 0.9905, train f1: 0.9905, val loss: 2136.25, val acc: 0.9881, val f1: 0.9881, time: 979.33s


100%|██████████| 26078/26078 [12:04<00:00, 35.99it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9956    0.9900    0.9928      4780
           D     0.9702    0.9717    0.9710      1274
           O     0.9995    0.9992    0.9993     56815
           P     0.9989    0.9986    0.9987     11997
           S     0.9971    0.9962    0.9967     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9839    0.9963    0.9900      4292

    accuracy                         0.9980     99611
   macro avg     0.9921    0.9931    0.9926     99611
weighted avg     0.9980    0.9980    0.9980     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9987    0.9659    0.9820      1641
           D     0.9767    0.9472    0.9617       398
           O     0.9926    0.9998    0.9962     18985
           P     0.9990    0.9975    0.9982      3936
           S     0.9985    0.9895    0.9940      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:41<00:00, 85.42it/s]


Epoch:2, Training loss: 4113.76, train acc: 0.9980, train f1: 0.9980, val loss: 2118.95, val acc: 0.9947, val f1: 0.9947, time: 979.82s


100%|██████████| 500/500 [00:02<00:00, 187.43it/s]


0.9947232715716255

#### aspects 2+3
#### FastText SG + Domain Embedding

In [ ]:
word_emb_fast_sg = word_embedding_table(word_list, fasttext_sg = True)
domain_emb_ex = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = True)
embedding_dim_concate23 = {'word': word_emb_fast_sg.shape[1], 'domain': domain_emb_ex.shape[1]}
embedding_dic_concate23 = {'word': word_emb_fast_sg, 'domain': domain_emb_ex}
emb_domain_f1_concate23, emb_domain_test_concate23 = bi_lstm_crf_attention(embedding_dim_concate23, embedding_dic_concate23, tag_to_ix, params, indicators,
                                                                           train_input_index, train_input_char_index, train_output_index,
                                                                           val_input_index, val_input_char_index,
                                                                           test_input_index, test_input_char_index)
emb_domain_f1_concate23

100%|██████████| 26078/26078 [11:51<00:00, 36.64it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9863    0.9958    0.9910      4780
           D     0.9480    0.9451    0.9465      1274
           O     0.9943    0.9979    0.9961     56815
           P     0.9999    0.9982    0.9990     11997
           S     0.9984    0.9901    0.9942     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9928    0.9595    0.9758      4292

    accuracy                         0.9949     99611
   macro avg     0.9885    0.9838    0.9861     99611
weighted avg     0.9949    0.9949    0.9949     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9665    0.9842    0.9752      1641
           D     0.9436    0.9246    0.9340       398
           O     0.9916    0.9954    0.9935     18985
           P     0.9995    0.9972    0.9983      3936
           S     0.9982    0.9874    0.9927      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:41<00:00, 85.40it/s] 


Epoch:1, Training loss: 23202.18, train acc: 0.9949, train f1: 0.9949, val loss: 1224.05, val acc: 0.9920, val f1: 0.9920, time: 967.02s


100%|██████████| 26078/26078 [11:59<00:00, 36.25it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9950    0.9975    0.9962      4780
           D     0.9950    0.9435    0.9686      1274
           O     0.9984    0.9985    0.9984     56815
           P     0.9998    0.9947    0.9972     11997
           S     0.9989    0.9954    0.9972     10035
        SEPA     0.9994    1.0000    0.9997     10418
           T     0.9599    0.9918    0.9756      4292

    accuracy                         0.9968     99611
   macro avg     0.9924    0.9888    0.9904     99611
weighted avg     0.9969    0.9968    0.9968     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9800    0.9866    0.9833      1641
           D     0.9865    0.9196    0.9519       398
           O     0.9947    0.9954    0.9951     18985
           P     0.9985    0.9939    0.9962      3936
           S     0.9988    0.9901    0.9944      3322
        SEPA     0.9994 

100%|██████████| 8705/8705 [01:43<00:00, 84.37it/s]


Epoch:2, Training loss: 2661.27, train acc: 0.9968, train f1: 0.9968, val loss: 1347.38, val acc: 0.9928, val f1: 0.9928, time: 978.17s


100%|██████████| 500/500 [00:02<00:00, 186.64it/s]


0.9928344426455598

#### aspects 1+3

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
domain_emb_ex = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = True)
embedding_dim_concate13 = {'pos_tag': postag_emb.shape[1], 'domain': domain_emb_ex.shape[1]}
embedding_dic_concate13 = {'pos_tag': postag_emb, 'domain': domain_emb_ex}

emb_domain_f1_concate13, emb_domain_test_concate13 = bi_lstm_crf_attention(embedding_dim_concate13, embedding_dic_concate13, tag_to_ix, params, indicators,
                                                                           train_input_index, train_input_char_index, train_output_index,
                                                                           val_input_index, val_input_char_index,
                                                                           test_input_index, test_input_char_index)
emb_domain_f1_concate13

100%|██████████| 26078/26078 [11:53<00:00, 36.55it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9778    0.9952    0.9864      4780
           D     0.9188    0.9498    0.9340      1274
           O     0.9949    0.9973    0.9961     56815
           P     0.9991    0.9977    0.9984     11997
           S     0.9979    0.9906    0.9942     10035
        SEPA     0.9999    0.9997    0.9998     10418
           T     0.9993    0.9592    0.9788      4292

    accuracy                         0.9946     99611
   macro avg     0.9839    0.9842    0.9840     99611
weighted avg     0.9946    0.9946    0.9946     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9557    0.9860    0.9706      1641
           D     0.9115    0.9322    0.9217       398
           O     0.9919    0.9949    0.9934     18985
           P     1.0000    0.9972    0.9986      3936
           S     0.9976    0.9874    0.9924      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:41<00:00, 85.96it/s]


Epoch:1, Training loss: 22465.16, train acc: 0.9946, train f1: 0.9946, val loss: 1354.30, val acc: 0.9917, val f1: 0.9917, time: 968.16s


100%|██████████| 26078/26078 [11:45<00:00, 36.98it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9981    0.9969    0.9975      4780
           D     0.9459    0.9882    0.9666      1274
           O     0.9993    0.9986    0.9990     56815
           P     0.9998    0.9995    0.9997     11997
           S     0.9973    0.9979    0.9976     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9942    0.9916    0.9929      4292

    accuracy                         0.9983     99611
   macro avg     0.9907    0.9961    0.9933     99611
weighted avg     0.9983    0.9983    0.9983     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9914    0.9817    0.9865      1641
           D     0.8773    0.9698    0.9212       398
           O     0.9948    0.9959    0.9954     18985
           P     0.9997    0.9992    0.9995      3936
           S     0.9967    0.9934    0.9950      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:31<00:00, 57.32it/s]


Epoch:2, Training loss: 2422.43, train acc: 0.9983, train f1: 0.9983, val loss: 1033.12, val acc: 0.9943, val f1: 0.9943, time: 1010.85s


100%|██████████| 500/500 [00:06<00:00, 81.32it/s]


0.9942735503987528

### Full -- Best Model (2stacked bilstm)
#### Pos Embedding + Word Embedding (Glove + FastText SG +Character)+ Domain Embedding

In [ ]:
# when testing embeddings, fixed other parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 1,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}


In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
word_emb = word_embedding_table(word_list, concate = True)
domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = False, concate = True)

embedding_dim_full = {'pos_tag': postag_emb.shape[1], 
                                      'word': word_emb.shape[1], 
                                      'char':params["char_emb"],
                                      'domain': domain_emb.shape[1]}
embedding_dic_full = {'pos_tag': postag_emb,
                                      'word': word_emb,
                                      'domain': domain_emb}

emb_word_full, emb_word_full_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full

[==================================================] 100.0% 387.1/387.1MB downloaded


100%|██████████| 26078/26078 [14:31<00:00, 29.93it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9977    0.9933    0.9955      4780
           D     0.9923    0.9050    0.9466      1274
           O     0.9944    0.9980    0.9962     56815
           P     0.9994    0.9984    0.9989     11997
           S     0.9944    0.9931    0.9938     10035
        SEPA     0.9997    1.0000    0.9999     10418
           T     0.9851    0.9737    0.9794      4292

    accuracy                         0.9953     99611
   macro avg     0.9947    0.9802    0.9872     99611
weighted avg     0.9953    0.9953    0.9952     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9907    0.9775    0.9840      1641
           D     0.9889    0.8920    0.9379       398
           O     0.9915    0.9959    0.9937     18985
           P     0.9995    0.9985    0.9990      3936
           S     0.9928    0.9895    0.9911      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:00<00:00, 72.03it/s]


Epoch:1, Training loss: 20412.54, train acc: 0.9953, train f1: 0.9953, val loss: 1159.36, val acc: 0.9926, val f1: 0.9926, time: 1209.32s


...

Train classification report
              precision    recall  f1-score   support

           C     0.9979    0.9985    0.9982      4780
           D     0.9960    0.9710    0.9833      1274
           O     0.9980    0.9998    0.9989     56815
           P     1.0000    0.9992    0.9996     11997
           S     0.9993    0.9977    0.9985     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9972    0.9860    0.9916      4292

    accuracy                         0.9985     99611
   macro avg     0.9983    0.9932    0.9957     99611
weighted avg     0.9985    0.9985    0.9985     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9908    0.9817    0.9862      1641
           D     0.9868    0.9422    0.9640       398
           O     0.9939    0.9978    0.9958     18985
           P     0.9995    0.9992    0.9994      3936
           S     0.9964    0.9931    0.9947      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:00<00:00, 72.37it/s]


Epoch:2, Training loss: 2481.03, train acc: 0.9985, train f1: 0.9985, val loss: 910.61, val acc: 0.9951, val f1: 0.9951, time: 1207.60s


100%|██████████| 500/500 [00:03<00:00, 130.25it/s]


0.9950830485099238

## Ablation Study - Attention Strategy

### Attention position

#### 2 stacked bilstm blocks (4 layers), attention between Stacked bilstm
-- Embedding -- stacked 2 layers Bilstm block -- attention --stacked 2 layers Bilstm block 

In [ ]:
# when testing embeddings, fixed other parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
word_emb = word_embedding_table(word_list, concate = True)
domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = False, concate = True)

embedding_dim_full = {'pos_tag': postag_emb.shape[1], 
                                      'word': word_emb.shape[1], 
                                      'char':params["char_emb"],
                                      'domain': domain_emb.shape[1]}
embedding_dic_full = {'pos_tag': postag_emb,
                                      'word': word_emb,
                                      'domain': domain_emb}

In [ ]:
class Model_attentionbefore(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embeddings_dict, hidden_dim, nheads, stacked_layer, tag_to_ix, attention_type, num_layers, indicators):
        """
        This model put attention between stacked bilstm layers.
        """
        super(Model_attentionbefore, self).__init__()
        self.bilstm = BiLSTM(vocab_size, embedding_dim, embeddings_dict, hidden_dim, len(tag_to_ix), num_layers, indicators)
        self.attention = MultiHeadAttention(hidden_dim, nheads, attention_type)
        self.crf = CRF(tag_to_ix)
        self.indicators =  indicators
        self.hidden_dim = hidden_dim
        self.hidden2tag = nn.Linear(hidden_dim, len(tag_to_ix))
        self.stacked_layer = stacked_layer
        if self.stacked_layer > 1:
            self.stacked_lstm = nn.ModuleList()
            for i in range(self.stacked_layer - 1): # removing the first layer
                self.stacked_lstm.append(nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=num_layers, batch_first = True, bidirectional=True).to(device))
                
        if indicators["is_crf"] == True:
          self.loss_func = None # if CRF is true, then loss func is None
        else:
          self.loss_func = nn.CrossEntropyLoss()

    def forward(self, sentence, words, tags):
        lstm_feats = self.bilstm(sentence, words).to(device)
        feats = lstm_feats
        if self.indicators["is_attention"] == True:
            output = self.attention(lstm_feats)
            feats = output
            feats = feats.view(-1, self.hidden_dim).to(device)
        if self.stacked_layer > 1:
          for layer in self.stacked_lstm:
            feats, _ = layer(feats)
        feats = feats.view(-1, self.hidden_dim).to(device)
        feats = self.hidden2tag(feats).to(device)
        return self.loss(feats, tags)

    def loss(self, feats, tags):
        if self.indicators["is_crf"] == True:
          return self.crf.neg_log_likelihood(feats, tags)
        return self.loss_func(feats,tags)

    def predict(self, sentence, words):
        lstm_feats = self.bilstm(sentence, words).to(device)
        feats = lstm_feats
        if  self.indicators["is_attention"] == True:
            output = self.attention(lstm_feats)            
            feats = output
            feats = feats.view(-1, self.hidden_dim).to(device)
        if self.stacked_layer > 1:
            for layer in self.stacked_lstm:
              feats, _ = layer(feats)
              
        if  self.indicators["is_crf"] == True:
             # Find the best path, given the features.
            feats = feats.view(-1, self.hidden_dim).to(device)
            feats = self.hidden2tag(feats).to(device)
            # print(feats.shape)
            score, tag_seq = self.crf._viterbi_decode(feats)   
            return score, tag_seq

        # else no crf layer
        feats = feats.view(-1, self.hidden_dim).to(device)
        feats = self.hidden2tag(feats).to(device)

        pred = feats.argmax(dim =1).tolist()
        assert(len(pred) == len(sentence))
        score = None
        return score, pred

In [ ]:
import datetime
from tqdm import tqdm

def bi_lstm_crf_attention_before(embedding_dim, embedding_dic, tag_to_ix, params, indicators, 
                          train_input_index, train_input_char_index,
                          train_output_index,
                          val_input_index, val_input_char_index,
                          test_input_index, test_input_char_index):

  hidden_dim = params["hidden_dim"]
  nheads = params["nheads"]
  vocab_size = params['vocab_size']
  learning_rate = params['lr']
  attention_type = params['attention_type']
  num_layers = params['num_layers']
  stacked_layer = params['stacked_layer']
  model = Model_attentionbefore(vocab_size, embedding_dim, embedding_dic, hidden_dim, nheads, 
                stacked_layer, tag_to_ix, attention_type, num_layers, 
                indicators).to(device)
  print(model)
  optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  EPOCH = 2
  for epoch in range(EPOCH):  
      time1 = datetime.datetime.now()
      train_loss = 0
      model.train()
      for i, idxs in enumerate(tqdm(train_input_index)):
          tags_index = train_output_index[i]
          # Step 1. Remember that Pytorch accumulates gradients.
          # We need to clear them out before each instance
          model.zero_grad()
          # Step 2. Get our inputs ready for the network, that is,
          # turn them into Tensors of word indices.
          sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
          sentence_words = torch.tensor(padding(train_input_char_index[i]), dtype=torch.long).to(device)
          targets = torch.tensor(tags_index, dtype=torch.long).to(device)

          # Step 3. Run our forward pass.
          loss = model(sentence_in, sentence_words, targets)
          # Step 4. Compute the loss, gradients, and update the parameters by
          # calling optimizer.step()
          loss.backward()
          optimizer.step()
          train_loss+=loss.item()
      model.eval()
      # Call the cal_acc functions you implemented as required
      predicted_train, ground_truth_train, train_acc = cal_acc(model,train_input_index, train_input_char_index, train_output_index)
      predicted_val, ground_truth_val, val_acc= cal_acc(model,val_input_index, val_input_char_index, val_output_index)
      predicted_train_tag = [ix_to_tag[x] for x in predicted_train]
      truth_train_tag = [ix_to_tag[x] for x in ground_truth_train]
      predicted_val_tag = [ix_to_tag[x] for x in predicted_val]
      truth_val_tag = [ix_to_tag[x] for x in ground_truth_val]
      print("Train classification report")
      print(classification_report(truth_train_tag, predicted_train_tag, digits = 4))
      train_f1 = f1_score(truth_train_tag, predicted_train_tag, average='micro')
    
      print("Val classification report")
      print(classification_report(truth_val_tag, predicted_val_tag, digits = 4))
      val_f1 = f1_score(truth_val_tag, predicted_val_tag, average='micro')

      val_loss = 0
      for i, idxs in enumerate(tqdm(val_input_index)):
          tags_index = val_output_index[i]
          sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
          sentence_words = torch.tensor(padding(val_input_char_index[i]), dtype=torch.long).to(device)
          targets = torch.tensor(tags_index, dtype=torch.long).to(device)
          loss = model(sentence_in, sentence_words, targets)
          val_loss+=loss.item()
      time2 = datetime.datetime.now()

      print("Epoch:%d, Training loss: %.2f, train acc: %.4f, train f1: %.4f, val loss: %.2f, val acc: %.4f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, train_f1, val_loss, val_acc, val_f1, (time2-time1).total_seconds()))
  
  test_predicted = generate_test(model, test_input_index, test_input_char_index)
  return val_f1, test_predicted

In [ ]:
emb_word_full_att_before, emb_word_full_test_att_before = bi_lstm_crf_attention_before(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_att_before

Model_attentionbefore(
  (bilstm): BiLSTM(
    (postag_word_embeds): Embedding(11243, 45)
    (semantic_word_embeds): Embedding(11243, 200)
    (char_embs): Embedding(60, 256)
    (char_lstm): LSTM(256, 128, batch_first=True, bidirectional=True)
    (domain_word_embeds): Embedding(11243, 100)
    (dropout): Dropout(p=0.2, inplace=False)
    (layernorm): LayerNorm((601,), eps=1e-05, elementwise_affine=True)
    (word_lstm): LSTM(601, 256, num_layers=2, batch_first=True, bidirectional=True)
    (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  )
  (attention): MultiHeadAttention(
    (attention_dropout): Dropout(p=0.2, inplace=False)
    (query): Linear(in_features=512, out_features=512, bias=False)
    (key): Linear(in_features=512, out_features=512, bias=False)
    (value): Linear(in_features=512, out_features=512, bias=False)
  )
  (crf): CRF()
  (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  (stacked_lstm): ModuleList(
    (0): LSTM(512, 256, num_

100%|██████████| 26078/26078 [15:34<00:00, 27.92it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9608    0.9891    0.9747      4780
           D     0.9004    0.8587    0.8791      1274
           O     0.9921    0.9967    0.9944     56815
           P     0.9966    0.9974    0.9970     11997
           S     0.9964    0.9813    0.9888     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9897    0.9418    0.9651      4292

    accuracy                         0.9911     99611
   macro avg     0.9765    0.9664    0.9713     99611
weighted avg     0.9911    0.9911    0.9911     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9599    0.9781    0.9689      1641
           D     0.8302    0.8844    0.8564       398
           O     0.9903    0.9951    0.9927     18985
           P     0.9977    0.9977    0.9977      3936
           S     0.9988    0.9777    0.9881      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:05<00:00, 69.53it/s]


Epoch:1, Training loss: 30067.63, train acc: 0.9911, train f1: 0.9911, val loss: 1620.07, val acc: 0.9894, val f1: 0.9894, time: 1290.31s


...

Train classification report
              precision    recall  f1-score   support

           C     0.9862    0.9870    0.9866      4780
           D     0.9650    0.9513    0.9581      1274
           O     0.9965    0.9997    0.9981     56815
           P     0.9998    0.9981    0.9990     11997
           S     0.9962    0.9921    0.9942     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9938    0.9697    0.9816      4292

    accuracy                         0.9962     99611
   macro avg     0.9910    0.9854    0.9882     99611
weighted avg     0.9962    0.9962    0.9962     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9839    0.9689    0.9764      1641
           D     0.9635    0.9296    0.9463       398
           O     0.9924    0.9988    0.9956     18985
           P     1.0000    0.9980    0.9990      3936
           S     0.9958    0.9889    0.9923      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:24<00:00, 60.44it/s]


Epoch:2, Training loss: 3645.26, train acc: 0.9962, train f1: 0.9962, val loss: 1189.61, val acc: 0.9936, val f1: 0.9936, time: 1304.42s


100%|██████████| 500/500 [00:04<00:00, 123.11it/s]


0.9935539965221563

#### 2 stacked bilstm blocks (4 layers), attention after stacked bilstm 
-- Embedding -- stacked 2 layers Bilstm block  -- stacked 2 layer2 Bilstm block -- attention


In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full, emb_word_full_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full

100%|██████████| 26078/26078 [14:14<00:00, 30.53it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9894    0.9918    0.9906      4780
           D     0.9892    0.8666    0.9238      1274
           O     0.9924    0.9985    0.9954     56815
           P     0.9991    0.9979    0.9985     11997
           S     0.9993    0.9863    0.9928     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9854    0.9718    0.9785      4292

    accuracy                         0.9942     99611
   macro avg     0.9935    0.9733    0.9828     99611
weighted avg     0.9942    0.9942    0.9941     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9799    0.9805    0.9802      1641
           D     0.9861    0.8920    0.9367       398
           O     0.9905    0.9968    0.9936     18985
           P     0.9985    0.9985    0.9985      3936
           S     1.0000    0.9822    0.9910      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [01:59<00:00, 72.65it/s]


Epoch:1, Training loss: 21453.98, train acc: 0.9942, train f1: 0.9942, val loss: 1252.02, val acc: 0.9922, val f1: 0.9922, time: 1198.15s


100%|██████████| 26078/26078 [14:15<00:00, 30.49it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9952    0.9977    0.9964      4780
           D     0.9896    0.9725    0.9810      1274
           O     0.9979    0.9993    0.9986     56815
           P     0.9989    0.9996    0.9993     11997
           S     0.9998    0.9973    0.9986     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9981    0.9863    0.9921      4292

    accuracy                         0.9982     99611
   macro avg     0.9971    0.9932    0.9951     99611
weighted avg     0.9982    0.9982    0.9982     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9836    0.9848    0.9842      1641
           D     0.9569    0.9472    0.9520       398
           O     0.9948    0.9967    0.9958     18985
           P     0.9985    0.9997    0.9991      3936
           S     0.9979    0.9919    0.9949      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:02<00:00, 70.87it/s]


Epoch:2, Training loss: 2636.56, train acc: 0.9982, train f1: 0.9982, val loss: 979.81, val acc: 0.9949, val f1: 0.9949, time: 1206.95s


100%|██████████| 500/500 [00:04<00:00, 123.61it/s]


0.9948731786292498

### Attention calculation


#### attention after stacked bilstm before crf 

##### Scaled-Dot Product (refer to Attention position test 2)

##### Dot product 

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full_dot, emb_word_full_dot_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_dot

100%|██████████| 26078/26078 [14:29<00:00, 30.01it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9908    0.9912    0.9910      4780
           D     0.9381    0.9278    0.9329      1274
           O     0.9924    0.9981    0.9953     56815
           P     0.9991    0.9990    0.9990     11997
           S     0.9993    0.9891    0.9942     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9971    0.9483    0.9721      4292

    accuracy                         0.9941     99611
   macro avg     0.9881    0.9791    0.9835     99611
weighted avg     0.9941    0.9941    0.9941     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9739    0.9768    0.9754      1641
           D     0.9335    0.9171    0.9252       398
           O     0.9898    0.9962    0.9930     18985
           P     0.9982    0.9987    0.9985      3936
           S     0.9985    0.9859    0.9921      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:02<00:00, 71.30it/s]


Epoch:1, Training loss: 17908.33, train acc: 0.9941, train f1: 0.9941, val loss: 1167.76, val acc: 0.9916, val f1: 0.9916, time: 1215.32s


100%|██████████| 26078/26078 [14:37<00:00, 29.72it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9950    0.9958    0.9954      4780
           D     0.9992    0.9349    0.9659      1274
           O     0.9978    0.9989    0.9983     56815
           P     0.9999    0.9997    0.9998     11997
           S     0.9994    0.9980    0.9987     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9848    0.9930    0.9889      4292

    accuracy                         0.9978     99611
   macro avg     0.9966    0.9886    0.9924     99611
weighted avg     0.9978    0.9978    0.9978     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9907    0.9756    0.9831      1641
           D     0.9972    0.9095    0.9514       398
           O     0.9933    0.9972    0.9952     18985
           P     0.9997    0.9995    0.9996      3936
           S     0.9976    0.9928    0.9952      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:03<00:00, 70.28it/s]


Epoch:2, Training loss: 2186.21, train acc: 0.9978, train f1: 0.9978, val loss: 929.98, val acc: 0.9943, val f1: 0.9943, time: 1229.09s


100%|██████████| 500/500 [00:04<00:00, 123.51it/s]


0.9942735503987528

##### Cosine

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'cosine', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full_cosine, emb_word_full_cosine_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_cosine

100%|██████████| 26078/26078 [15:00<00:00, 28.95it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9950    0.9912    0.9931      4780
           D     0.9790    0.9160    0.9465      1274
           O     0.9948    0.9967    0.9957     56815
           P     1.0000    0.9982    0.9991     11997
           S     0.9975    0.9916    0.9946     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9629    0.9788    0.9708      4292

    accuracy                         0.9946     99611
   macro avg     0.9899    0.9818    0.9857     99611
weighted avg     0.9947    0.9946    0.9946     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9889    0.9793    0.9841      1641
           D     0.9735    0.9221    0.9471       398
           O     0.9925    0.9946    0.9936     18985
           P     1.0000    0.9985    0.9992      3936
           S     0.9973    0.9883    0.9927      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:06<00:00, 68.84it/s]


Epoch:1, Training loss: 19009.41, train acc: 0.9946, train f1: 0.9946, val loss: 1188.58, val acc: 0.9924, val f1: 0.9924, time: 1257.55s


100%|██████████| 26078/26078 [15:13<00:00, 28.54it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9964    0.9971    0.9968      4780
           D     0.9872    0.9710    0.9790      1274
           O     0.9985    0.9993    0.9989     56815
           P     0.9994    0.9997    0.9995     11997
           S     0.9995    0.9971    0.9983     10035
        SEPA     1.0000    0.9999    1.0000     10418
           T     0.9923    0.9911    0.9917      4292

    accuracy                         0.9984     99611
   macro avg     0.9962    0.9936    0.9949     99611
weighted avg     0.9984    0.9984    0.9984     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9908    0.9805    0.9856      1641
           D     0.9594    0.9497    0.9545       398
           O     0.9947    0.9968    0.9958     18985
           P     0.9995    0.9997    0.9996      3936
           S     0.9970    0.9922    0.9946      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:10<00:00, 66.48it/s]


Epoch:2, Training loss: 2880.23, train acc: 0.9984, train f1: 0.9984, val loss: 924.17, val acc: 0.9947, val f1: 0.9947, time: 1280.08s


100%|██████████| 500/500 [00:04<00:00, 117.23it/s]


0.9946932901601008

##### General

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'general', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full_general, emb_word_full_general_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_general

100%|██████████| 26078/26078 [15:55<00:00, 27.29it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9812    0.9956    0.9884      4780
           D     0.9744    0.9262    0.9497      1274
           O     0.9945    0.9977    0.9961     56815
           P     1.0000    0.9967    0.9984     11997
           S     0.9971    0.9929    0.9950     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9935    0.9685    0.9809      4292

    accuracy                         0.9951     99611
   macro avg     0.9915    0.9825    0.9869     99611
weighted avg     0.9951    0.9951    0.9951     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9643    0.9866    0.9753      1641
           D     0.9761    0.9246    0.9497       398
           O     0.9923    0.9954    0.9938     18985
           P     1.0000    0.9964    0.9982      3936
           S     0.9961    0.9889    0.9924      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:13<00:00, 65.13it/s]


Epoch:1, Training loss: 17877.91, train acc: 0.9951, train f1: 0.9951, val loss: 1277.19, val acc: 0.9926, val f1: 0.9926, time: 1324.95s


100%|██████████| 26078/26078 [15:10<00:00, 28.64it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9787    0.9983    0.9884      4780
           D     0.9866    0.9827    0.9847      1274
           O     0.9982    0.9979    0.9980     56815
           P     1.0000    0.9995    0.9997     11997
           S     0.9995    0.9976    0.9986     10035
        SEPA     1.0000    1.0000    1.0000     10418
           T     0.9969    0.9863    0.9916      4292

    accuracy                         0.9976     99611
   macro avg     0.9943    0.9946    0.9944     99611
weighted avg     0.9976    0.9976    0.9976     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9559    0.9902    0.9728      1641
           D     0.9476    0.9548    0.9512       398
           O     0.9953    0.9945    0.9949     18985
           P     0.9995    0.9992    0.9994      3936
           S     0.9985    0.9925    0.9955      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:06<00:00, 68.92it/s]


Epoch:2, Training loss: 2487.58, train acc: 0.9976, train f1: 0.9976, val loss: 1209.48, val acc: 0.9937, val f1: 0.9937, time: 1284.09s


100%|██████████| 500/500 [00:04<00:00, 122.37it/s]


0.9937338849913054

#### attention between stacked Bilstm before crf
Embedding -- bilstm -- attention -- bilstm -- crf

##### Scaled-Dot Product (refer to Attention position test 1)

##### Dot Product

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full_att_before_dot, emb_word_full_test_att_before_dot = bi_lstm_crf_attention_before(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_att_before_dot

Model_attentionbefore(
  (bilstm): BiLSTM(
    (postag_word_embeds): Embedding(11243, 45)
    (semantic_word_embeds): Embedding(11243, 200)
    (char_embs): Embedding(60, 256)
    (char_lstm): LSTM(256, 128, batch_first=True, bidirectional=True)
    (domain_word_embeds): Embedding(11243, 100)
    (dropout): Dropout(p=0.2, inplace=False)
    (layernorm): LayerNorm((601,), eps=1e-05, elementwise_affine=True)
    (word_lstm): LSTM(601, 256, num_layers=2, batch_first=True, bidirectional=True)
    (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  )
  (attention): MultiHeadAttention(
    (attention_dropout): Dropout(p=0.2, inplace=False)
    (query): Linear(in_features=512, out_features=512, bias=False)
    (key): Linear(in_features=512, out_features=512, bias=False)
    (value): Linear(in_features=512, out_features=512, bias=False)
  )
  (crf): CRF()
  (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  (stacked_lstm): ModuleList(
    (0): LSTM(512, 256, num_

100%|██████████| 26078/26078 [16:58<00:00, 25.61it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9568    0.9730    0.9648      4780
           D     0.9321    0.6138    0.7402      1274
           O     0.9859    0.9780    0.9820     56815
           P     0.9933    0.9968    0.9950     11997
           S     0.9765    0.9770    0.9767     10035
        SEPA     0.9839    0.9996    0.9917     10418
           T     0.7949    0.9049    0.8464      4292

    accuracy                         0.9744     99611
   macro avg     0.9462    0.9205    0.9281     99611
weighted avg     0.9753    0.9744    0.9743     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9514    0.9549    0.9532      1641
           D     0.8750    0.5980    0.7104       398
           O     0.9835    0.9777    0.9806     18985
           P     0.9942    0.9970    0.9956      3936
           S     0.9788    0.9750    0.9769      3322
        SEPA     0.9844 

100%|██████████| 8705/8705 [02:13<00:00, 65.27it/s]


Epoch:1, Training loss: 30899.17, train acc: 0.9744, train f1: 0.9744, val loss: 3612.07, val acc: 0.9733, val f1: 0.9733, time: 1386.30s


...

Train classification report
              precision    recall  f1-score   support

           C     0.7851    0.6697    0.7228      4780
           D     0.0000    0.0000    0.0000      1274
           O     0.9307    0.9277    0.9292     56815
           P     0.6209    0.8325    0.7113     11997
           S     0.8072    0.7400    0.7721     10035
        SEPA     0.9557    0.9962    0.9755     10418
           T     0.6718    0.4310    0.5251      4292

    accuracy                         0.8588     99611
   macro avg     0.6816    0.6567    0.6623     99611
weighted avg     0.8535    0.8588    0.8528     99611

Val classification report
              precision    recall  f1-score   support

           C     0.7822    0.6435    0.7061      1641
           D     0.0000    0.0000    0.0000       398
           O     0.9290    0.9258    0.9274     18985
           P     0.6180    0.8255    0.7068      3936
           S     0.7994    0.7414    0.7693      3322
        SEPA     0.9504 

100%|██████████| 8705/8705 [02:11<00:00, 66.17it/s]


Epoch:2, Training loss: 43361.99, train acc: 0.8588, train f1: 0.8588, val loss: 16297.94, val acc: 0.8565, val f1: 0.8565, time: 1345.49s


100%|██████████| 500/500 [00:04<00:00, 118.85it/s]


0.8564789830305211

##### Cosine

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'cosine', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full_att_before_cosine, emb_word_full_test_att_before_cosine = bi_lstm_crf_attention_before(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_att_before_cosine

Model_attentionbefore(
  (bilstm): BiLSTM(
    (postag_word_embeds): Embedding(11243, 45)
    (semantic_word_embeds): Embedding(11243, 200)
    (char_embs): Embedding(60, 256)
    (char_lstm): LSTM(256, 128, batch_first=True, bidirectional=True)
    (domain_word_embeds): Embedding(11243, 100)
    (dropout): Dropout(p=0.2, inplace=False)
    (layernorm): LayerNorm((601,), eps=1e-05, elementwise_affine=True)
    (word_lstm): LSTM(601, 256, num_layers=2, batch_first=True, bidirectional=True)
    (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  )
  (attention): MultiHeadAttention(
    (attention_dropout): Dropout(p=0.2, inplace=False)
    (query): Linear(in_features=512, out_features=512, bias=False)
    (key): Linear(in_features=512, out_features=512, bias=False)
    (value): Linear(in_features=512, out_features=512, bias=False)
  )
  (crf): CRF()
  (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  (stacked_lstm): ModuleList(
    (0): LSTM(512, 256, num_

100%|██████████| 26078/26078 [16:40<00:00, 26.07it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9827    0.9889    0.9858      4780
           D     0.9449    0.8077    0.8709      1274
           O     0.9913    0.9977    0.9945     56815
           P     0.9993    0.9983    0.9988     11997
           S     0.9900    0.9878    0.9889     10035
        SEPA     0.9998    1.0000    0.9999     10418
           T     0.9954    0.9548    0.9747      4292

    accuracy                         0.9923     99611
   macro avg     0.9862    0.9622    0.9734     99611
weighted avg     0.9922    0.9923    0.9922     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9687    0.9793    0.9739      1641
           D     0.8950    0.8141    0.8526       398
           O     0.9898    0.9952    0.9925     18985
           P     0.9995    0.9985    0.9990      3936
           S     0.9909    0.9859    0.9884      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:15<00:00, 64.20it/s]


Epoch:1, Training loss: 27851.17, train acc: 0.9923, train f1: 0.9923, val loss: 1676.83, val acc: 0.9901, val f1: 0.9901, time: 1378.72s


...

Train classification report
              precision    recall  f1-score   support

           C     0.9983    0.9866    0.9924      4780
           D     0.9738    0.9615    0.9676      1274
           O     0.9962    0.9995    0.9978     56815
           P     0.9926    0.9994    0.9960     11997
           S     0.9989    0.9929    0.9959     10035
        SEPA     0.9999    1.0000    1.0000     10418
           T     0.9961    0.9644    0.9800      4292

    accuracy                         0.9962     99611
   macro avg     0.9937    0.9863    0.9900     99611
weighted avg     0.9962    0.9962    0.9962     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9950    0.9732    0.9840      1641
           D     0.9741    0.9447    0.9592       398
           O     0.9926    0.9983    0.9955     18985
           P     0.9927    0.9992    0.9959      3936
           S     0.9985    0.9901    0.9943      3322
        SEPA     0.9997 

100%|██████████| 8705/8705 [02:11<00:00, 66.02it/s]


Epoch:2, Training loss: 4086.03, train acc: 0.9962, train f1: 0.9962, val loss: 1275.59, val acc: 0.9939, val f1: 0.9939, time: 1384.33s


100%|██████████| 500/500 [00:04<00:00, 119.31it/s]


0.9938538106374049

##### General

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'general', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_full_att_before_general, emb_word_full_test_att_before_general = bi_lstm_crf_attention_before(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_full_att_before_general

Model_attentionbefore(
  (bilstm): BiLSTM(
    (postag_word_embeds): Embedding(11243, 45)
    (semantic_word_embeds): Embedding(11243, 200)
    (char_embs): Embedding(60, 256)
    (char_lstm): LSTM(256, 128, batch_first=True, bidirectional=True)
    (domain_word_embeds): Embedding(11243, 100)
    (dropout): Dropout(p=0.2, inplace=False)
    (layernorm): LayerNorm((601,), eps=1e-05, elementwise_affine=True)
    (word_lstm): LSTM(601, 256, num_layers=2, batch_first=True, bidirectional=True)
    (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  )
  (attention): MultiHeadAttention(
    (attention_dropout): Dropout(p=0.2, inplace=False)
    (query): Linear(in_features=512, out_features=512, bias=False)
    (key): Linear(in_features=512, out_features=512, bias=False)
    (value): Linear(in_features=512, out_features=512, bias=False)
  )
  (crf): CRF()
  (hidden2tag): Linear(in_features=512, out_features=9, bias=True)
  (stacked_lstm): ModuleList(
    (0): LSTM(512, 256, num_

100%|██████████| 26078/26078 [16:25<00:00, 26.45it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.3243    0.0025    0.0050      4780
           D     0.0000    0.0000    0.0000      1274
           O     0.7574    0.9143    0.8285     56815
           P     0.4474    0.6463    0.5287     11997
           S     0.8093    0.4091    0.5435     10035
        SEPA     0.8231    0.6778    0.7434     10418
           T     0.0000    0.0000    0.0000      4292

    accuracy                         0.7116     99611
   macro avg     0.4516    0.3786    0.3784     99611
weighted avg     0.6690    0.7116    0.6689     99611

Val classification report
              precision    recall  f1-score   support

           C     0.4167    0.0030    0.0060      1641
           D     0.0000    0.0000    0.0000       398
           O     0.7578    0.9168    0.8298     18985
           P     0.4404    0.6423    0.5225      3936
           S     0.7985    0.4175    0.5483      3322
        SEPA     0.8402 

100%|██████████| 8705/8705 [02:07<00:00, 68.35it/s]


Epoch:1, Training loss: 67642.60, train acc: 0.7116, train f1: 0.7116, val loss: 30890.99, val acc: 0.7124, val f1: 0.7124, time: 1349.75s


...

Train classification report
              precision    recall  f1-score   support

           C     0.5000    0.0006    0.0013      4780
           D     0.0000    0.0000    0.0000      1274
           O     0.6323    0.7939    0.7039     56815
           P     0.2046    0.3467    0.2573     11997
           S     0.8480    0.3876    0.5321     10035
        SEPA     0.6132    0.1974    0.2986     10418
           T     0.0000    0.0000    0.0000      4292

    accuracy                         0.5543     99611
   macro avg     0.3997    0.2466    0.2562     99611
weighted avg     0.5588    0.5543    0.5174     99611

Val classification report
              precision    recall  f1-score   support

           C     0.0000    0.0000    0.0000      1641
           D     0.0000    0.0000    0.0000       398
           O     0.6314    0.8020    0.7066     18985
           P     0.1985    0.3323    0.2485      3936
           S     0.8522    0.3958    0.5406      3322
        SEPA     0.6208 

100%|██████████| 8705/8705 [02:02<00:00, 70.83it/s]


Epoch:2, Training loss: 104688.52, train acc: 0.5543, train f1: 0.5543, val loss: 37592.23, val acc: 0.5557, val f1: 0.5557, time: 1291.61s


100%|██████████| 500/500 [00:04<00:00, 124.15it/s]


0.5557054626131799

## Ablation Study - different Stacked layer
**default attention after stacked bilstm layers**

### Stacked 1 blocks (2 layers) (refer to full - best model)

### Stacked 2 blocks (4 layers) (refer to Attention position test 2)

### stacked 3 blocks (6 layers)

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 3,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
lens_restriction = True
if lens_restriction == True:
    domain_data_chat = chat_list_clean_len
else:
    domain_data_chat = chat_list_clean

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
word_emb = word_embedding_table(word_list, concate = True)
domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = False, concate = True)

embedding_dim_full = {'pos_tag': postag_emb.shape[1], 
                      'word': word_emb.shape[1], 
                      'char':params["char_emb"],
                      'domain': domain_emb.shape[1]}
embedding_dic_full = {'pos_tag': postag_emb,
                      'word': word_emb,
                      'domain': domain_emb}

In [ ]:
emb_word_stacked3, emb_word_stacked3_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_stacked3

100%|██████████| 26078/26078 [17:18<00:00, 25.12it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.7897    0.9571    0.8654      4780
           D     0.7064    0.0604    0.1114      1274
           O     0.9841    0.9967    0.9903     56815
           P     0.9997    0.9896    0.9946     11997
           S     0.9952    0.9742    0.9846     10035
        SEPA     0.9956    0.9997    0.9977     10418
           T     0.9908    0.9247    0.9566      4292

    accuracy                         0.9769     99611
   macro avg     0.9231    0.8432    0.8429     99611
weighted avg     0.9757    0.9769    0.9723     99611

Val classification report
              precision    recall  f1-score   support

           C     0.7979    0.9458    0.8656      1641
           D     0.7692    0.1005    0.1778       398
           O     0.9810    0.9951    0.9880     18985
           P     0.9997    0.9881    0.9939      3936
           S     0.9975    0.9711    0.9841      3322
        SEPA     0.9953 

100%|██████████| 8705/8705 [02:18<00:00, 63.05it/s]


Epoch:1, Training loss: 56127.88, train acc: 0.9769, train f1: 0.9769, val loss: 3432.74, val acc: 0.9756, val f1: 0.9756, time: 1432.29s


100%|██████████| 26078/26078 [17:33<00:00, 24.75it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9671    0.9895    0.9782      4780
           D     0.9326    0.8469    0.8877      1274
           O     0.9965    0.9977    0.9971     56815
           P     0.9954    0.9991    0.9973     11997
           S     0.9931    0.9936    0.9934     10035
        SEPA     0.9995    1.0000    0.9998     10418
           T     0.9981    0.9711    0.9844      4292

    accuracy                         0.9942     99611
   macro avg     0.9832    0.9711    0.9768     99611
weighted avg     0.9942    0.9942    0.9941     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9719    0.9683    0.9701      1641
           D     0.9111    0.8492    0.8791       398
           O     0.9928    0.9968    0.9948     18985
           P     0.9932    0.9995    0.9963      3936
           S     0.9940    0.9892    0.9916      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [02:20<00:00, 62.10it/s]


Epoch:2, Training loss: 7709.03, train acc: 0.9942, train f1: 0.9942, val loss: 1840.66, val acc: 0.9919, val f1: 0.9919, time: 1449.85s


100%|██████████| 500/500 [00:04<00:00, 111.27it/s]


0.9919050188882893

### Stacked 4 blocks (8 layers)

In [ ]:
indicators = {'is_attention': True, "is_crf": True, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 4,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_stacked4, emb_word_stacked4_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_stacked4

100%|██████████| 26078/26078 [18:12<00:00, 23.87it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.5014    0.0362    0.0675      4780
           D     0.0000    0.0000    0.0000      1274
           O     0.8670    0.9988    0.9283     56815
           P     0.9976    0.9538    0.9752     11997
           S     0.7771    0.8440    0.8092     10035
        SEPA     0.9921    0.9603    0.9759     10418
           T     0.2572    0.0815    0.1238      4292

    accuracy                         0.8753     99611
   macro avg     0.6275    0.5535    0.5543     99611
weighted avg     0.8319    0.8753    0.8391     99611

Val classification report
              precision    recall  f1-score   support

           C     0.6019    0.0396    0.0743      1641
           D     0.0000    0.0000    0.0000       398
           O     0.8648    0.9987    0.9270     18985
           P     0.9976    0.9505    0.9735      3936
           S     0.7697    0.8453    0.8057      3322
        SEPA     0.9908 

100%|██████████| 8705/8705 [02:21<00:00, 61.50it/s]


Epoch:1, Training loss: 102069.40, train acc: 0.8753, train f1: 0.8753, val loss: 12321.94, val acc: 0.8740, val f1: 0.8740, time: 1501.61s


100%|██████████| 26078/26078 [18:01<00:00, 24.11it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.7873    0.5197    0.6261      4780
           D     0.0000    0.0000    0.0000      1274
           O     0.9844    0.9973    0.9908     56815
           P     0.9997    0.9944    0.9971     11997
           S     0.9845    0.9822    0.9833     10035
        SEPA     0.9968    0.9978    0.9973     10418
           T     0.5457    0.8292    0.6582      4292

    accuracy                         0.9526     99611
   macro avg     0.7569    0.7601    0.7504     99611
weighted avg     0.9466    0.9526    0.9470     99611

Val classification report
              precision    recall  f1-score   support

           C     0.7803    0.4784    0.5931      1641
           D     0.0000    0.0000    0.0000       398
           O     0.9800    0.9966    0.9882     18985
           P     0.9997    0.9924    0.9960      3936
           S     0.9801    0.9795    0.9798      3322
        SEPA     0.9969 

100%|██████████| 8705/8705 [02:22<00:00, 60.95it/s]


Epoch:2, Training loss: 25054.48, train acc: 0.9526, train f1: 0.9526, val loss: 5711.73, val acc: 0.9487, val f1: 0.9487, time: 1491.12s


100%|██████████| 500/500 [00:04<00:00, 105.44it/s]


0.9486718234694489

## Ablation Study - with/without CRF

### previous section with crf (refer to Attention position test 2)

### no crf

In [ ]:
postag_emb = pos_embedding_table(tag_index, word_list)
word_emb = word_embedding_table(word_list, concate = True)
domain_emb = domain_emb_table(domain_to_ix_glossary, domain_data_chat, word_list, external = False, concate = True)

embedding_dim_full = {'pos_tag': postag_emb.shape[1], 
                      'word': word_emb.shape[1], 
                      'char':params["char_emb"],
                      'domain': domain_emb.shape[1]}
embedding_dic_full = {'pos_tag': postag_emb,
                      'word': word_emb,
                      'domain': domain_emb}

In [ ]:
indicators = {'is_attention': True, "is_crf": False, "is_layernorm": True, "is_dropout": False}
params = {"attention_type": 'scaled-dot', "char_emb": 256, "hidden_dim": 512, 'nheads': 8,
          'lr':0.05, 'num_layers': 2, 'stacked_layer': 2,
          "vocab_size": {'word': len(word_list), 'char': len(char_list)}}

In [ ]:
emb_word_no_crf, emb_word_stacked_no_crf_test = bi_lstm_crf_attention(embedding_dim_full, embedding_dic_full, tag_to_ix, 
                                                          params, indicators,
                                                          train_input_index, train_input_char_index,
                                                          train_output_index,
                                                          val_input_index, val_input_char_index,
                                                          test_input_index, test_input_char_index)
emb_word_no_crf

100%|██████████| 26078/26078 [05:20<00:00, 81.33it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9738    0.9707    0.9722      4780
           D     0.9843    0.4922    0.6562      1274
           O     0.9777    0.9986    0.9880     56815
           P     0.9987    0.9966    0.9976     11997
           S     0.9968    0.9712    0.9838     10035
        SEPA     0.9993    1.0000    0.9997     10418
           T     0.9928    0.9264    0.9584      4292

    accuracy                         0.9848     99611
   macro avg     0.9890    0.9079    0.9366     99611
weighted avg     0.9849    0.9848    0.9837     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9644    0.9580    0.9612      1641
           D     0.9721    0.5251    0.6819       398
           O     0.9774    0.9972    0.9872     18985
           P     0.9972    0.9975    0.9973      3936
           S     0.9963    0.9687    0.9823      3322
        SEPA     0.9992 

100%|██████████| 8705/8705 [00:38<00:00, 227.51it/s]


Epoch:1, Training loss: 8084.31, train acc: 0.9848, train f1: 0.9848, val loss: 525.08, val acc: 0.9839, val f1: 0.9839, time: 486.98s


100%|██████████| 26078/26078 [05:10<00:00, 84.04it/s]


Train classification report
              precision    recall  f1-score   support

           C     0.9927    0.9937    0.9932      4780
           D     0.9855    0.9082    0.9453      1274
           O     0.9943    0.9990    0.9967     56815
           P     0.9981    0.9985    0.9983     11997
           S     0.9983    0.9941    0.9962     10035
        SEPA     0.9997    1.0000    0.9999     10418
           T     0.9983    0.9658    0.9818      4292

    accuracy                         0.9957     99611
   macro avg     0.9953    0.9799    0.9873     99611
weighted avg     0.9957    0.9957    0.9957     99611

Val classification report
              precision    recall  f1-score   support

           C     0.9770    0.9829    0.9800      1641
           D     0.9623    0.8970    0.9285       398
           O     0.9918    0.9967    0.9942     18985
           P     0.9992    0.9992    0.9992      3936
           S     0.9967    0.9889    0.9927      3322
        SEPA     1.0000 

100%|██████████| 8705/8705 [00:37<00:00, 229.90it/s]


Epoch:2, Training loss: 1055.76, train acc: 0.9957, train f1: 0.9957, val loss: 300.66, val acc: 0.9930, val f1: 0.9930, time: 481.73s


100%|██████████| 500/500 [00:02<00:00, 232.65it/s]


0.9929543682916592